In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Environmental settings and magic commands

In [3]:
%cd ../../micropython/ports/unix/

/home/v923z/sandbox/micropython/v1.11/micropython/ports/unix


In [4]:
from IPython.core.magic import Magics, magics_class, line_cell_magic
from IPython.core.magic import cell_magic, register_cell_magic, register_line_magic
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring
import subprocess
import os

## micropython magic command

The following magic class takes the content of a cell, and depending on the arguments, either passes it to the unix, or the stm32 implementation. In the latter case, a pyboard must be connected to the computer, and must be initialised beforehand. 

In [5]:
@magics_class
class PyboardMagic(Magics):
    @cell_magic
    @magic_arguments()
    @argument('-skip')
    @argument('-unix')
    @argument('-file')
    @argument('-data')
    @argument('-time')
    @argument('-memory')
    def micropython(self, line='', cell=None):
        args = parse_argstring(self.micropython, line)
        if args.skip: # doesn't care about the cell's content
            print('skipped execution')
            return None # do not parse the rest
        if args.unix: # tests the code on the unix port. Note that this works on unix only
            with open('/dev/shm/micropython.py', 'w') as fout:
                fout.write(cell)
            proc = subprocess.Popen(["./micropython", "/dev/shm/micropython.py"], 
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print(proc.stdout.read().decode("utf-8"))
            print(proc.stderr.read().decode("utf-8"))
            return None
        if args.file: # can be used to copy the cell content onto the pyboard's flash
            spaces = "    "
            try:
                with open(args.file, 'w') as fout:
                    fout.write(cell.replace('\t', spaces))
                    print('written cell to {}'.format(args.file))
            except:
                print('Failed to write to disc!')
            return None # do not parse the rest
        if args.data: # can be used to load data from the pyboard directly into kernel space
            message = pyb.exec(cell)
            if len(message) == 0:
                print('pyboard >>>')
            else:
                print(message.decode('utf-8'))
                # register new variable in user namespace
                self.shell.user_ns[args.data] = string_to_matrix(message.decode("utf-8"))
        
        if args.time: # measures the time of executions
            pyb.exec('import utime')
            message = pyb.exec('t = utime.ticks_us()\n' + cell + '\ndelta = utime.ticks_diff(utime.ticks_us(), t)' + 
                               "\nprint('execution time: {:d} us'.format(delta))")
            print(message.decode('utf-8'))
        
        if args.memory: # prints out memory information 
            message = pyb.exec('from micropython import mem_info\nprint(mem_info())\n')
            print("memory before execution:\n========================\n", message.decode('utf-8'))
            message = pyb.exec(cell)
            print(">>> ", message.decode('utf-8'))
            message = pyb.exec('print(mem_info())')
            print("memory after execution:\n========================\n", message.decode('utf-8'))

        else:
            message = pyb.exec(cell)
            print(message.decode('utf-8'))

ip = get_ipython()
ip.register_magics(PyboardMagic)

## Code magic

The following cell magic simply writes a licence header, and the contents of the cell to the file given in the header of the cell. 

In [6]:
@magics_class
class MyMagics(Magics):
        
    @cell_magic
    def ccode(self, line, cell):
        copyright = """/*
 * This file is part of the micropython-ulab project, 
 *
 * https://github.com/v923z/micropython-ulab
 *
 * The MIT License (MIT)
 *
 * Copyright (c) 2019 Zoltán Vörös
*/
    """
        if line:
            with open('../../../ulab/code/'+line, 'w') as cout:
                cout.write(copyright)
                cout.write(cell)
            print('written %d bytes to %s'%(len(copyright) + len(cell), line))
            return None

ip = get_ipython()
ip.register_magics(MyMagics)

# The ndarray type

## General comments

`ndarrays` are efficient containers of numerical data of the same type (i.e., signed/unsigned chars, signed/unsigned integers or `float`s, which, depending on the platform, can also be `double`s). Beyond storing the actual data, the type definition has a couple of additional members (on top of the `base` type). Namely, the number of dimensions (`size_t`), the shape (`*size_t`), the strides (`*int32_t`), the offset of the data pointer (`size_t`), the total length of the array (`size_t`), and a `uint8_t`, which determines, whether the arrays is to be treated as a set of Booleans.
The type definition is as follows:

```c
typedef struct _ndarray_obj_t {
    mp_obj_base_t base;
    uint8_t boolean;
    uint8_t ndim;
    size_t *shape;
    int32_t *strides;
    size_t offset;
    size_t len;
    mp_obj_array_t *array;
} ndarray_obj_t;
```

## ndarray.h

In [428]:
%%ccode ndarray.h

#ifndef _NDARRAY_
#define _NDARRAY_

#include "py/objarray.h"
#include "py/binary.h"
#include "py/objstr.h"
#include "py/objlist.h"

#define SWAP(t, a, b) { t tmp = a; a = b; b = tmp; }

#define NDARRAY_NUMERIC   0
#define NDARRAY_BOOLEAN   1

#if MICROPY_FLOAT_IMPL == MICROPY_FLOAT_IMPL_FLOAT
#define FLOAT_TYPECODE 'f'
#elif MICROPY_FLOAT_IMPL == MICROPY_FLOAT_IMPL_DOUBLE
#define FLOAT_TYPECODE 'd'
#endif

const mp_obj_type_t ulab_ndarray_type;

enum NDARRAY_TYPE {
    NDARRAY_BOOL = '?', // this must never be assigned to the typecode!
    NDARRAY_UINT8 = 'B',
    NDARRAY_INT8 = 'b',
    NDARRAY_UINT16 = 'H', 
    NDARRAY_INT16 = 'h',
    NDARRAY_FLOAT = FLOAT_TYPECODE,
};

typedef struct _ndarray_obj_t {
    mp_obj_base_t base;
    uint8_t boolean;
    uint8_t ndim;
    size_t *shape;
    int32_t *strides;
    size_t len;
    size_t offset;
    mp_obj_array_t *array;
} ndarray_obj_t;

// this is a helper structure, so that we can return shape AND strides from a function
typedef struct _ndarray_header_obj_t {
    size_t *shape;
    int32_t *strides;
    int8_t axis;
    size_t offset;
} ndarray_header_obj_t;

// various helper functions
size_t ndarray_index_from_flat(size_t , ndarray_obj_t *, int32_t *);
size_t ndarray_index_from_contracted(size_t  , ndarray_obj_t * , int32_t * , uint8_t  , uint8_t  );
mp_float_t ndarray_get_float_value(void *, uint8_t , size_t );

// calculates the index (in the original linear array) of an item, if the index in the flat array is given
// this is the macro equivalent of ndarray_index_from_flat()
// TODO: This fails, when the last stride is not 1!!!
#define NDARRAY_INDEX_FROM_FLAT(ndarray, shape_strides, index, _tindex, _nindex) do {\
    size_t Q;\
    (_tindex) = (index);\
    (_nindex) = (ndarray)->offset;\
    for(size_t _x=0; _x < (ndarray)->ndim; _x++) {\
        Q = (_tindex) / (shape_strides)[_x];\
        (_tindex) -= Q * (shape_strides)[_x];\
        (_nindex) += Q * (ndarray)->strides[_x];\
    }\
} while(0)

#define NDARRAY_INDEX_FROM_FLAT2(ndarray, stride_array, shape_strides, index, _tindex, _nindex) do {\
    size_t Q;\
    (_tindex) = (index);\
    (_nindex) = (ndarray)->offset;\
    for(size_t _x=0; _x < (ndarray)->ndim; _x++) {\
        Q = (_tindex) / (shape_strides)[_x];\
        (_tindex) -= Q * (shape_strides)[_x];\
        (_nindex) += Q * (stride_array)[_x];\
    }\
} while(0)
    
#define CREATE_SINGLE_ITEM(ndarray, type, typecode, value) do {\
    (ndarray) = ndarray_new_linear_array(1, (typecode));\
    type *tmparr = (type *)(ndarray)->array->items;\
    tmparr[0] = (type)(value);\
} while(0)

#define RUN_BINARY_LOOP(ndarray, typecode, type_out, type_left, type_right, lhs, rhs, shape, ndim, operator) do {\
    uint8_t *left = (uint8_t *)(lhs)->array->items;\
    uint8_t *right = (uint8_t *)(rhs)->array->items;\
    (ndarray) = ndarray_new_dense_ndarray((ndim), (shape), (typecode));\
    uint8_t size_left = sizeof(type_left), size_right = sizeof(type_right);\
    type_out *out = (type_out *)ndarray->array->items;\
    for(size_t i=0; i < (lhs)->len; i++) {\
        out[i] = *(type_left *)left + *(type_right *)right;\
        left += size_left; right += size_right;\
    }\
} while(0)

mp_obj_t mp_obj_new_ndarray_iterator(mp_obj_t , size_t , mp_obj_iter_buf_t *);
void ndarray_print(const mp_print_t *, mp_obj_t , mp_print_kind_t );
ndarray_obj_t *ndarray_new_ndarray(uint8_t , size_t *, int32_t *, uint8_t );
ndarray_obj_t *ndarray_new_dense_ndarray(uint8_t , size_t *, uint8_t );
ndarray_obj_t *ndarray_new_linear_array(size_t , uint8_t );
ndarray_obj_t *ndarray_copy_view(ndarray_obj_t *, uint8_t );

mp_obj_t ndarray_copy(mp_obj_t );
mp_obj_t ndarray_make_new(const mp_obj_type_t *, size_t , size_t , const mp_obj_t *);
mp_obj_t ndarray_subscr(mp_obj_t , mp_obj_t , mp_obj_t );
mp_obj_t ndarray_getiter(mp_obj_t , mp_obj_iter_buf_t *);
mp_obj_t ndarray_binary_op(mp_binary_op_t , mp_obj_t , mp_obj_t );
mp_obj_t ndarray_unary_op(mp_unary_op_t , mp_obj_t );

mp_obj_t ndarray_shape(mp_obj_t );
mp_obj_t ndarray_reshape(mp_obj_t , mp_obj_t );
mp_obj_t ndarray_transpose(mp_obj_t );
mp_obj_t ndarray_flatten(size_t , const mp_obj_t *, mp_map_t *);
mp_obj_t ndarray_itemsize(mp_obj_t );
mp_obj_t ndarray_strides(mp_obj_t );
mp_obj_t ndarray_info(mp_obj_t );

#endif

written 4464 bytes to ndarray.h


## ndarray.c

In [459]:
%%ccode ndarray.c

#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "py/runtime.h"
#include "py/binary.h"
#include "py/obj.h"
#include "py/objtuple.h"
#include "ndarray.h"

// This function is copied verbatim from objarray.c
STATIC mp_obj_array_t *array_new(char typecode, size_t n) {
    int typecode_size = mp_binary_get_size('@', typecode, NULL);
    mp_obj_array_t *o = m_new_obj(mp_obj_array_t);
    // this step could probably be skipped: we are never going to store a bytearray per se
    #if MICROPY_PY_BUILTINS_BYTEARRAY && MICROPY_PY_ARRAY
    o->base.type = (typecode == BYTEARRAY_TYPECODE) ? &mp_type_bytearray : &mp_type_array;
    #elif MICROPY_PY_BUILTINS_BYTEARRAY
    o->base.type = &mp_type_bytearray;
    #else
    o->base.type = &mp_type_array;
    #endif
    o->typecode = typecode;
    o->free = 0;
    o->len = n;
    o->items = m_new(byte, typecode_size * o->len);
    return o;
}

// helper functions
mp_float_t ndarray_get_float_value(void *data, uint8_t typecode, size_t index) {
    if(typecode == NDARRAY_UINT8) {
        return (mp_float_t)((uint8_t *)data)[index];
    } else if(typecode == NDARRAY_INT8) {
        return (mp_float_t)((int8_t *)data)[index];
    } else if(typecode == NDARRAY_UINT16) {
        return (mp_float_t)((uint16_t *)data)[index];
    } else if(typecode == NDARRAY_INT16) {
        return (mp_float_t)((int16_t *)data)[index];
    } else {
        return (mp_float_t)((mp_float_t *)data)[index];
    }
}

size_t ndarray_index_from_contracted(size_t index, ndarray_obj_t *ndarray, int32_t *strides, uint8_t ndim, uint8_t axis) {
    // calculates the index in the original (linear) array from the index in the contracted (linear) array
    size_t q, new_index = 0;
    for(size_t i=0; i <= ndim-1; i++) {
        q = index / strides[i];
        if(i < axis) { 
            new_index += q * ndarray->strides[i];
        } else {
            new_index += q * ndarray->strides[i+1];            
        }
        index -= q * strides[i];
    }
    return new_index + ndarray->offset;
}

size_t *ndarray_new_coords(uint8_t ndim) {
    size_t *coords = m_new(size_t, ndim);
    memset(coords, 0, ndim*sizeof(size_t));
    return coords;
}

void ndarray_print(const mp_print_t *print, mp_obj_t self_in, mp_print_kind_t kind) {
    (void)kind;
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    size_t offset = self->offset;
    uint8_t print_extra = self->ndim;
    size_t *coords = ndarray_new_coords(self->ndim);
    
    mp_print_str(print, "array(");
        
    for(size_t i=0; i < self->len; i++) {
        for(uint8_t j=0; j < print_extra; j++) {
            mp_print_str(print, "[");
        }
        print_extra = 0;
        if(!self->boolean) {
            mp_obj_print_helper(print, mp_binary_get_val_array(self->array->typecode, self->array->items, offset), PRINT_REPR);
        } else {
            if(((uint8_t *)self->array->items)[offset]) {
                mp_print_str(print, "True");
            } else {
                mp_print_str(print, "False");
            }
        }
        offset += self->strides[self->ndim-1];
        coords[self->ndim-1] += 1;
        if(coords[self->ndim-1] != self->shape[self->ndim-1]) {
            mp_print_str(print, ", ");
        }
        for(uint8_t j=self->ndim-1; j > 0; j--) {
            if(coords[j] == self->shape[j]) {
                offset -= self->shape[j] * self->strides[j];
                offset += self->strides[j-1];
                print_extra += 1;
                coords[j] = 0;
                coords[j-1] += 1;
                mp_print_str(print, "]");
            } else { // coordinates can change only, if the last coordinate changes
                break;
            }
        }
        if(print_extra && (i != self->len-1)) {
            mp_print_str(print, ",\n");
            if(print_extra > 1) {
                mp_print_str(print, "\n");
            }
        }
    }
    mp_print_str(print, "]");
    m_del(size_t, coords, self->ndim);

    if(self->boolean) {
        mp_print_str(print, ", dtype=bool)");
    } else if(self->array->typecode == NDARRAY_UINT8) {
        mp_print_str(print, ", dtype=uint8)");
    } else if(self->array->typecode == NDARRAY_INT8) {
        mp_print_str(print, ", dtype=int8)");
    } else if(self->array->typecode == NDARRAY_UINT16) {
        mp_print_str(print, ", dtype=uint16)");
    } else if(self->array->typecode == NDARRAY_INT16) {
        mp_print_str(print, ", dtype=int16)");
    } else if(self->array->typecode == NDARRAY_FLOAT) {
        mp_print_str(print, ", dtype=float)");
    }
}

ndarray_obj_t *ndarray_new_ndarray(uint8_t ndim, size_t *shape, int32_t *strides, uint8_t typecode) {
    // Creates the base ndarray with shape, and initialises the values to straight 0s
    ndarray_obj_t *ndarray = m_new_obj(ndarray_obj_t);
    ndarray->base.type = &ulab_ndarray_type;
    ndarray->ndim = ndim;
    ndarray->shape = shape;
    ndarray->strides = strides;
    ndarray->offset = 0;
    if(typecode == NDARRAY_BOOL) {
        typecode = NDARRAY_UINT8;
        ndarray->boolean = NDARRAY_BOOLEAN;
    } else {
        ndarray->boolean = NDARRAY_NUMERIC;
    }
    ndarray->len = 1;
    for(uint8_t i=0; i < ndim; i++) {
        ndarray->len *= shape[i];
    }
    mp_obj_array_t *array = array_new(typecode, ndarray->len);
    // this should set all elements to 0, irrespective of the of the typecode (all bits are zero)
    // we could, perhaps, leave this step out, and initialise the array only, when needed
    memset(array->items, 0, array->len); 
    ndarray->array = array;
    return ndarray;
}

ndarray_obj_t *ndarray_new_dense_ndarray(uint8_t ndim, size_t *shape, uint8_t typecode) {
    // creates a dense array, i.e., one, where the strides are derived directly from the shapes
    int32_t *strides = m_new(int32_t, ndim);
    strides[ndim-1] = 1;
    for(size_t i=ndim-1; i > 0; i--) {
        strides[i-1] = strides[i] * shape[i];
    }
    return ndarray_new_ndarray(ndim, shape, strides, typecode);
}

ndarray_obj_t *ndarray_new_view(mp_obj_array_t *array, uint8_t ndim, size_t *shape, int32_t *strides, size_t offset, uint8_t boolean) {
    ndarray_obj_t *ndarray = m_new_obj(ndarray_obj_t);
    ndarray->base.type = &ulab_ndarray_type;
    ndarray->boolean = boolean;
    ndarray->ndim = ndim;
    ndarray->shape = shape;
    ndarray->strides = strides;
    ndarray->len = 1;
    for(uint8_t i=0; i < ndim; i++) {
        ndarray->len *= shape[i];
    }    
    ndarray->offset = offset;
    ndarray->array = array;
    return ndarray;
}

ndarray_obj_t *ndarray_copy_view(ndarray_obj_t *input, uint8_t typecode) {
    // Creates a new ndarray from the input
    // If the input was a sliced view, the output will inherit the shape, but not the strides

    int32_t *strides = m_new(int32_t, input->ndim);
    strides[input->ndim-1] = 1;
    for(uint8_t i=input->ndim-1; i > 0; i--) {
        strides[i-1] = strides[i] * input->shape[i];
    }
    ndarray_obj_t *ndarray = ndarray_new_ndarray(input->ndim, input->shape, strides, typecode);
    ndarray->boolean = input->boolean;
    
    mp_obj_t item;
    size_t offset = input->offset;
    size_t *coords = ndarray_new_coords(input->ndim);
    
    for(size_t i=0; i < ndarray->len; i++) {
        item = mp_binary_get_val_array(input->array->typecode, input->array->items, offset);
        mp_binary_set_val_array(typecode, ndarray->array->items, i, item);
        offset += input->strides[input->ndim-1];
        coords[input->ndim-1] += 1;
        for(uint8_t j=ndarray->ndim-1; j > 0; j--) {
            if(coords[j] == input->shape[j]) {
                offset -= input->shape[j] * input->strides[j];
                offset += input->strides[j-1];
                coords[j] = 0;
                coords[j-1] += 1;
            } else { // coordinates can change only, if the last coordinate changes
                break;
            }
        }
    }
    m_del(size_t, coords, input->ndim);
    return ndarray;
}

ndarray_obj_t *ndarray_new_linear_array(size_t len, uint8_t dtype) {
    size_t *shape = m_new(size_t, 1);
    int32_t *strides = m_new(int32_t, 1);
    shape[0] = len;
    strides[0] = 1;
    return ndarray_new_ndarray(1, shape, strides, dtype);
}

STATIC uint8_t ndarray_init_helper(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    static const mp_arg_t allowed_args[] = {
        { MP_QSTR_, MP_ARG_REQUIRED | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_none_obj)} },
        { MP_QSTR_dtype, MP_ARG_KW_ONLY | MP_ARG_INT, {.u_int = NDARRAY_FLOAT } },
    };
    
    mp_arg_val_t args[MP_ARRAY_SIZE(allowed_args)];
    mp_arg_parse_all(1, pos_args, kw_args, MP_ARRAY_SIZE(allowed_args), allowed_args, args);
    
    uint8_t dtype = args[1].u_int;
    // at this point, dtype can still be `?` for Boolean arrays
    return dtype;
}

mp_obj_t ndarray_make_new(const mp_obj_type_t *type, size_t n_args, size_t n_kw, const mp_obj_t *args) {
    // TODO: implement dtype, and copy keywords
    mp_arg_check_num(n_args, n_kw, 1, 2, true);
    mp_map_t kw_args;
    mp_map_init_fixed_table(&kw_args, n_kw, args + n_args);
    uint8_t dtype = ndarray_init_helper(n_args, args, &kw_args);

    mp_obj_t len_in = mp_obj_len_maybe(args[0]);
    size_t len = MP_OBJ_SMALL_INT_VALUE(len_in);
    ndarray_obj_t *self, *ndarray;
    
    if(MP_OBJ_IS_TYPE(args[0], &ulab_ndarray_type)) {
        ndarray = MP_OBJ_TO_PTR(args[0]);
        self = ndarray_copy_view(ndarray, dtype);
        return MP_OBJ_FROM_PTR(self);
    }
    // work with a single dimension for now
    self = ndarray_new_linear_array(len, dtype);
    
    size_t i=0;
    mp_obj_iter_buf_t iter_buf;
    mp_obj_t item, iterable = mp_getiter(args[0], &iter_buf);
    if(self->boolean) {
        uint8_t *array = (uint8_t *)self->array->items;
        while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
            if(mp_obj_get_float(item)) {
                *array = 1;
            }
            array++;
        }
    } else {
        while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
            mp_binary_set_val_array(dtype, self->array->items, i++, item);
        }
    }
    return MP_OBJ_FROM_PTR(self);
}

mp_bound_slice_t generate_slice(mp_uint_t n, mp_obj_t index) {
    // micropython seems to have difficulties with negative steps
    mp_bound_slice_t slice;
    if(MP_OBJ_IS_TYPE(index, &mp_type_slice)) {
        mp_seq_get_fast_slice_indexes(n, index, &slice);
    } else if(mp_obj_is_int(index)) {
        int32_t _index = mp_obj_get_int(index);
        if(_index < 0) {
            _index += n;
        } 
        if((_index >= n) || (_index < 0)) {
            mp_raise_msg(&mp_type_IndexError, "index is out of bounds");
        }
        slice.start = _index;
        slice.stop = _index + 1;
        slice.step = 1;
    } else {
        mp_raise_msg(&mp_type_IndexError, "indices must be integers, slices, or Boolean lists");
    }
    return slice;
}

size_t slice_length(mp_bound_slice_t slice) {
    int32_t len, correction = 1;
    if(slice.step > 0) correction = -1;
    len = (slice.stop - slice.start + (slice.step + correction)) / slice.step;
    if(len < 0) return 0;
    return (size_t)len;
}

ndarray_obj_t *ndarray_new_view_from_tuple(ndarray_obj_t *ndarray, mp_obj_tuple_t *slices) {
    mp_bound_slice_t slice;
    size_t *shape_array = m_new(size_t, ndarray->ndim);
    int32_t *strides_array = m_new(int32_t, ndarray->ndim);
    size_t offset = ndarray->offset;
    for(uint8_t i=0; i < ndarray->ndim; i++) {
        if(i < slices->len) {
            slice = generate_slice(ndarray->shape[i], slices->items[i]);
            offset += ndarray->offset + slice.start * ndarray->strides[i];
            shape_array[i] = slice_length(slice);
            strides_array[i] = ndarray->strides[i] * slice.step;
        } else {
            shape_array[i] = ndarray->shape[i];
            strides_array[i] = ndarray->strides[i]; 
        }
    }
    return ndarray_new_view(ndarray->array, ndarray->ndim, shape_array, strides_array, offset, ndarray->boolean);
}
    
bool ndarray_check_compatibility(ndarray_obj_t *lhs, ndarray_obj_t *rhs) {
    if(rhs->ndim > lhs->ndim) {
        return false;
    } 
    for(uint8_t i=0; i < rhs->ndim; i++) {
        if((rhs->shape[rhs->ndim-1-i] != 1) && (rhs->shape[rhs->ndim-1-i] != lhs->shape[lhs->ndim-1-i])) {
            return false;
        }
    }
    return true;
}

mp_obj_t ndarray_assign_view_from_tuple(ndarray_obj_t *ndarray, mp_obj_tuple_t *slices, mp_obj_t value) {
    ndarray_obj_t *lhs = ndarray_new_view_from_tuple(ndarray, slices);
    ndarray_obj_t *rhs;
    if(MP_OBJ_IS_TYPE(value, &ulab_ndarray_type)) {
        rhs = MP_OBJ_TO_PTR(value);
        // since this is an assignment, the left hand side should definitely be able to contain the right hand side
        if(!ndarray_check_compatibility(lhs, rhs)) {
            mp_raise_ValueError("could not broadcast input array into output array");
        }
    } else { // we have a scalar, so create an ndarray for it
        size_t *shape = m_new(size_t, lhs->ndim*sizeof(size_t));
        for(uint8_t i=0; i < lhs->ndim; i++) {
            shape[i] = 1;
        }
        rhs = ndarray_new_dense_ndarray(lhs->ndim, shape, lhs->array->typecode);
        mp_binary_set_val_array(rhs->array->typecode, rhs->array->items, 0, value);
    }
    size_t roffset = rhs->offset;
    size_t loffset = lhs->offset;
    size_t *lcoords = ndarray_new_coords(lhs->ndim);
    mp_obj_t item;
    uint8_t diff_ndim = lhs->ndim - rhs->ndim;
    for(size_t i=0; i < lhs->len; i++) {
        item = mp_binary_get_val_array(rhs->array->typecode, rhs->array->items, roffset);
        mp_binary_set_val_array(lhs->array->typecode, lhs->array->items, loffset, item);
        for(uint8_t j=lhs->ndim-1; j > 0; j--) {
            loffset += lhs->strides[j];
            lcoords[j] += 1;
            if(j >= diff_ndim) {
                if(rhs->shape[j-diff_ndim] != 1) {
                    roffset += rhs->strides[j-diff_ndim];
                }
            }
            if(lcoords[j] == lhs->shape[j]) { // we are at a dimension boundary
                if(j > diff_ndim) { // this means right-hand-side coordinates that haven't been prepended
                    if(rhs->shape[j-diff_ndim] != 1) {
                        // if rhs->shape[j-diff_ndim] != 1, then rhs->shape[j-diff_ndim] == lhs->shape[j],
                        // so we can advance the offset counter
                        roffset -= rhs->shape[j-diff_ndim] * rhs->strides[j-diff_ndim];
                        roffset += rhs->strides[j-diff_ndim-1];
                    } else { // rhs->shape[j-diff_ndim] == 1
                        roffset -= rhs->strides[j-diff_ndim];
                    }
                } else {
                    roffset = rhs->offset;
                }
                loffset -= lhs->shape[j] * lhs->strides[j];
                loffset += lhs->strides[j-1];
                lcoords[j] = 0;
                lcoords[j-1] += 1;
            } else { // coordinates can change only, if the last coordinate changes
                break;
            }
        }
    }
    m_del(size_t, lcoords, lhs->ndim);
    return mp_const_none;
}

mp_obj_t ndarray_subscr(mp_obj_t self, mp_obj_t index, mp_obj_t value) {
    ndarray_obj_t *ndarray = MP_OBJ_TO_PTR(self);
    
    if (value == MP_OBJ_SENTINEL) { // return value(s)
        if(mp_obj_is_int(index) || MP_OBJ_IS_TYPE(index, &mp_type_slice)) {
            mp_obj_t *items = m_new(mp_obj_t, 1);
            items[0] = index;
            mp_obj_t tuple = mp_obj_new_tuple(1, items);
            return ndarray_new_view_from_tuple(ndarray, tuple);
        }
        // first, check, whether all members of the tuple are integer scalars, or slices
        if(MP_OBJ_IS_TYPE(index, &mp_type_tuple)) {
            mp_obj_tuple_t *tuple = MP_OBJ_TO_PTR(index);
            for(uint8_t i=0; i < tuple->len; i++) {
                if(!MP_OBJ_IS_TYPE(tuple->items[i], &mp_type_slice) && !mp_obj_is_int(tuple->items[i])) {
                    // TODO: we have to return a copy here
                    mp_raise_msg(&mp_type_IndexError, "wrong index type");
                }
            }
            // now we know that we can return a view
            ndarray_obj_t *result = ndarray_new_view_from_tuple(ndarray, tuple);
            return MP_OBJ_FROM_PTR(result);
        }
    } else { // assignment; the value must be an ndarray, or a scalar
        if(!MP_OBJ_IS_TYPE(value, &ulab_ndarray_type) && !mp_obj_is_int(value) && !mp_obj_is_float(value)) {
            mp_raise_ValueError("right hand side must be an ndarray, or a scalar");
        } else {
            if(mp_obj_is_int(index) || MP_OBJ_IS_TYPE(index, &mp_type_slice)) {
                mp_obj_t *items = m_new(mp_obj_t, 1);
                items[0] = index;
                mp_obj_t tuple = mp_obj_new_tuple(1, items);
                return ndarray_assign_view_from_tuple(ndarray, tuple, value);
            } 
            if(MP_OBJ_IS_TYPE(index, &mp_type_tuple)) {
                mp_obj_tuple_t *tuple = MP_OBJ_TO_PTR(index);
                for(uint8_t i=0; i < tuple->len; i++) {
                    if(!MP_OBJ_IS_TYPE(tuple->items[i], &mp_type_slice) && !mp_obj_is_int(tuple->items[i])) {
                        // TODO: we have to return a copy here
                        mp_raise_msg(&mp_type_IndexError, "wrong index type");
                    }
                }
                return ndarray_assign_view_from_tuple(ndarray, tuple, value);
            } else {
                mp_raise_NotImplementedError("wrong index type");
            }
        }
    }
    return mp_const_none;
}

// itarray iterator
mp_obj_t ndarray_getiter(mp_obj_t o_in, mp_obj_iter_buf_t *iter_buf) {
    return mp_obj_new_ndarray_iterator(o_in, 0, iter_buf);
}

typedef struct _mp_obj_ndarray_it_t {
    mp_obj_base_t base;
    mp_fun_1_t iternext;
    mp_obj_t ndarray;
    size_t cur;
} mp_obj_ndarray_it_t;

mp_obj_t ndarray_iternext(mp_obj_t self_in) {
    mp_obj_ndarray_it_t *self = MP_OBJ_TO_PTR(self_in);
    ndarray_obj_t *ndarray = MP_OBJ_TO_PTR(self->ndarray);
    size_t iter_end = ndarray->shape[0];
    if(self->cur < iter_end) {
        if(ndarray->ndim == 1) { // we have a linear array
            // read the current value
            self->cur++;
            return mp_binary_get_val_array(ndarray->array->typecode, ndarray->array->items, self->cur-1);
        } else { // we have a tensor, return the reduced view
            size_t offset = ndarray->offset + self->cur * ndarray->strides[0];
            ndarray_obj_t *value = ndarray_new_view(ndarray->array, ndarray->ndim-1, ndarray->shape+1, ndarray->strides+1, offset, ndarray->boolean);
            self->cur++;
            return MP_OBJ_FROM_PTR(value);
        }
    } else {
        return MP_OBJ_STOP_ITERATION;
    }
}

mp_obj_t mp_obj_new_ndarray_iterator(mp_obj_t ndarray, size_t cur, mp_obj_iter_buf_t *iter_buf) {
    assert(sizeof(mp_obj_ndarray_it_t) <= sizeof(mp_obj_iter_buf_t));
    mp_obj_ndarray_it_t *o = (mp_obj_ndarray_it_t*)iter_buf;
    o->base.type = &mp_type_polymorph_iter;
    o->iternext = ndarray_iternext;
    o->ndarray = ndarray;
    o->cur = cur;
    return MP_OBJ_FROM_PTR(o);
}

mp_obj_t ndarray_shape(mp_obj_t self_in) {
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    mp_obj_t *items = m_new(mp_obj_t, self->ndim);
    size_t *shape = (size_t *)self->shape;
    for(uint8_t i=0; i < self->ndim; i++) {
        items[i] = mp_obj_new_int(shape[i]);
    }
    mp_obj_t tuple = mp_obj_new_tuple(self->ndim, items);
    m_del(mp_obj_t, items, self->ndim);
    return tuple;
}

mp_obj_t ndarray_strides(mp_obj_t self_in) {
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    mp_obj_t *items = m_new(mp_obj_t, self->ndim);
    int32_t *strides = (int32_t *)self->strides;
    for(int8_t i=0; i < self->ndim; i++) {
        items[i] = mp_obj_new_int(strides[i]);
    }
    mp_obj_t tuple = mp_obj_new_tuple(self->ndim, items);
    m_del(mp_obj_t, items, self->ndim);
    return tuple;
}

mp_obj_t ndarray_info(mp_obj_t self_in) {
    // TODO: the proper way of handling this would be to use mp_print_str()
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    printf("class: ndarray\n");
    printf("shape: (%ld", (size_t)self->shape[0]);
    for(uint8_t i=1; i < self->ndim; i++) {
        printf(", %ld", (size_t)self->shape[i]);
    }
    printf(")");
    printf("\nstrides: (%ld", (size_t)self->strides[0]);
    for(uint8_t i=1; i < self->ndim; i++) {
        printf(", %ld", (size_t)self->strides[i]);
    }
    printf(")");
    printf("\nitemsize: %ld\n", mp_binary_get_size('@', self->array->typecode, NULL));
    printf("data pointer: %p\n", self->array->items);
    if(self->array->typecode == NDARRAY_BOOL) {
        printf("type: bool\n");
    } else if(self->array->typecode == NDARRAY_UINT8) {
        printf("type: uint8\n");
    } else if(self->array->typecode == NDARRAY_INT8) {
        printf("type: int8\n");
    } else if(self->array->typecode == NDARRAY_UINT16) {
        printf("type: uint16\n");
    } else if(self->array->typecode == NDARRAY_INT16) {
        printf("type: int16\n");
    } else {
        printf("type: float\n");
    }
    return mp_const_none;
}

mp_obj_t ndarray_flatten(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    static const mp_arg_t allowed_args[] = {
        { MP_QSTR_order, MP_ARG_KW_ONLY | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_QSTR(MP_QSTR_C)} },
    };

    mp_arg_val_t args[MP_ARRAY_SIZE(allowed_args)];
    mp_arg_parse_all(n_args - 1, pos_args + 1, kw_args, MP_ARRAY_SIZE(allowed_args), allowed_args, args);
    ndarray_obj_t *ndarray = MP_OBJ_TO_PTR(pos_args[0]);
    
    GET_STR_DATA_LEN(args[0].u_obj, order, clen);
    if((clen != 1) || ((memcmp(order, "C", 1) != 0) && (memcmp(order, "F", 1) != 0))) {
        mp_raise_ValueError("flattening order must be either 'C', or 'F'");        
    }
    ndarray_obj_t *result = ndarray_new_linear_array(ndarray->len, ndarray->array->typecode);
    uint8_t itemsize = mp_binary_get_size('@', ndarray->array->typecode, NULL);
    if(ndarray->len == ndarray->array->len) { // this is a dense array, we can simply copy everything
        if(memcmp(order, "C", 1) == 0) { // C order; this should be fast, because no re-ordering is required
            memcpy(result->array->items, ndarray->array->items, itemsize*ndarray->len);
        } else { // Fortran order
            mp_raise_NotImplementedError("flatten is implemented in C order only");
        }
        return MP_OBJ_FROM_PTR(result);
    } else {
        uint8_t *rarray = (uint8_t *)result->array->items;
        uint8_t *narray = (uint8_t *)ndarray->array->items;
        size_t *coords = ndarray_new_coords(ndarray->ndim);

        size_t offset = ndarray->offset;
        if(memcmp(order, "C", 1) == 0) { // C order; this is a view, so we have to collect the items
            for(size_t i=0; i < result->len; i++) {
                memcpy(rarray, &narray[offset*itemsize], itemsize);
                rarray += itemsize;
                offset += ndarray->strides[ndarray->ndim-1];
                coords[ndarray->ndim-1] += 1;
                for(uint8_t j=ndarray->ndim-1; j > 0; j--) {
                    if(coords[j] == ndarray->shape[j]) {
                        offset -= ndarray->shape[j] * ndarray->strides[j];
                        offset += ndarray->strides[j-1];
                        coords[j] = 0;
                        coords[j-1] += 1;
                    } else { // coordinates can change only, if the last coordinate changes
                        break;
                    }
                }
            }
            m_del(size_t, coords, ndarray->ndim);
        } else { // Fortran order
            mp_raise_NotImplementedError("flatten is implemented for C order only");
        }
        //m_del(int32_t, shape_strides, 1);
        return MP_OBJ_FROM_PTR(result);
    }
}

mp_obj_t ndarray_reshape(mp_obj_t oin, mp_obj_t new_shape) {
    // TODO: not all reshaping operations can be realised via views!
    // returns a new view with the specified shape
    ndarray_obj_t *ndarray_in = MP_OBJ_TO_PTR(oin);
    mp_obj_tuple_t *shape = MP_OBJ_TO_PTR(new_shape);
    if(ndarray_in->len != ndarray_in->array->len) {
        mp_raise_ValueError("input and output shapes are not compatible");
    }
    size_t *shape_array = m_new(size_t, shape->len);
    int32_t *strides_array = m_new(int32_t, shape->len);
    size_t new_offset = ndarray_in->offset; // this has to be re-calculated
    strides_array[shape->len-1] = 1;
    shape_array[shape->len-1] = mp_obj_get_int(shape->items[shape->len-1]);
    for(uint8_t i=shape->len-1; i > 0; i--) {
        shape_array[i-1] = mp_obj_get_int(shape->items[i-1]);
        strides_array[i-1] = strides_array[i] * shape_array[i];
    }
    ndarray_obj_t *ndarray = ndarray_new_view(ndarray_in->array, shape->len, shape_array, strides_array, new_offset, ndarray_in->boolean);
    return MP_OBJ_FROM_PTR(ndarray);
}

mp_obj_t ndarray_transpose(mp_obj_t self_in) {
    // TODO: check, what happens to the offset here!
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    size_t *shape = m_new(size_t, self->ndim);
    int32_t *strides = m_new(int32_t, self->ndim);
    for(uint8_t i=0; i < self->ndim; i++) {
        shape[i] = self->shape[self->ndim-1-i];
        strides[i] = self->strides[self->ndim-1-i];
    }
    ndarray_obj_t *ndarray = ndarray_new_view(self->array, self->ndim, shape, strides, self->offset, self->boolean);
    return MP_OBJ_FROM_PTR(ndarray);
}

mp_obj_t ndarray_itemsize(mp_obj_t self_in) {
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    return mp_obj_new_int(mp_binary_get_size('@', self->array->typecode, NULL));
}

uint8_t upcasting(ndarray_obj_t *in1, ndarray_obj_t *in2) {
    if((in1->array->typecode = NDARRAY_FLOAT) || (in2->array->typecode = NDARRAY_FLOAT)) { // 9 cases
        return NDARRAY_FLOAT;
    } else if(in1->array->typecode == in2->array->typecode) { // 4 cases
        return in1->array->typecode;
    } else if(in1->array->typecode == NDARRAY_UINT8) { // 3 cases
        if(in2->array->typecode == NDARRAY_UINT16) return NDARRAY_UINT16;
        else return NDARRAY_INT16; // in2->array->typecode == NDARRAY_INT8, NDARRAY_INT16, 
    } else if(in1->array->typecode == NDARRAY_INT8) { // 3 cases
        return NDARRAY_INT16; // in2->array->typecode == NDARRAY_UINT8, NDARRAY_UINT16, NDARRAY_INT16
    } else if(in1->array->typecode == NDARRAY_UINT16) { // 3 cases
        if((in2->array->typecode == NDARRAY_UINT8) || (in2->array->typecode == NDARRAY_INT8)) return NDARRAY_UINT16;
        return NDARRAY_FLOAT; // in2->array->typecode == NDARRAY_INT16
    } else if(in1->array->typecode == NDARRAY_INT16) { // 3 cases
        if((in2->array->typecode == NDARRAY_UINT8) || (in2->array->typecode == NDARRAY_INT8)) return NDARRAY_INT16;
        return NDARRAY_FLOAT; // in2->array->typecode == NDARRAY_UINT16
    }
}

size_t *broadcasting(ndarray_obj_t *in1, ndarray_obj_t *in2) {
    // creates a new dense array with the dimensions
    // and shapes dictated by the broadcasting rules
    uint8_t ndim = in2->ndim;
    if(in1->ndim > in2->ndim) { 
        // overwrite ndim, if the first array is larger than the second
        ndim = in1->ndim;
    }
    size_t *shape = m_new(size_t, ndim);
    size_t *shape1 = m_new(size_t, ndim);
    size_t *shape2 = m_new(size_t, ndim);
    for(uint8_t i=0; i < ndim; i++) {
        // initialise the shapes with straight ones
        shape1[i] = shape2[i] = 1;
        shape[i] = 0;
    }
    // overwrite the first in1->ndim (in2->ndim) shape values from the right
    for(uint8_t i=ndim; i > ndim-in1->ndim; i--) {
        shape1[i-1] = in1->shape[i-1];
    }
    for(uint8_t i=ndim; i > ndim-in2->ndim; i--) {
        shape2[i-1] = in2->shape[i-1];
    }
    // check, whether the new shapes conform to the broadcasting rules
    for(uint8_t i=0; i < ndim; i++) {
        if((shape1[i] == shape2[i]) || (shape1[i] == 1) || (shape2[i] == 1)) {
            shape[i] = shape1[i];
            if(shape1[i] < shape2[i]) {
                shape[i] = shape2[i];
            }
        } else {
            m_del(size_t, shape1, ndim);
            m_del(size_t, shape2, ndim);
            break;
        }
    }
    m_del(size_t, shape1, ndim);
    m_del(size_t, shape2, ndim);
    return shape;
}

ndarray_obj_t *ndarray_do_binary_op(ndarray_obj_t *lhs, ndarray_obj_t *rhs, uint8_t op) {
    uint8_t typecode = upcasting(lhs, rhs);
    size_t *shape = broadcasting(lhs, rhs);
    uint8_t ndim = lhs->ndim;
    if(rhs->ndim > lhs->ndim) ndim = rhs->ndim;
    // this is going to be the result array
    ndarray_obj_t *ndarray = ndarray_new_dense_ndarray(ndim, shape, typecode);

    size_t offset = ndarray->offset;
    size_t roffset = rhs->offset;
    size_t loffset = lhs->offset;
    size_t *coords = ndarray_new_coords(ndim); // coordinates in ndarray
    size_t *lcoords = ndarray_new_coords(ndim); // coordinates in the left-hand-side ndarray
    size_t *rcoords = ndarray_new_coords(ndim); // coordinates in the right-hand-side ndarray
    int32_t *lstride = m_new(int32_t, ndim);
    int32_t *rstride = m_new(int32_t, ndim);
    for(uint8_t i=0; i < lhs->ndim; i++) {
        lstride[ndim-1-i] = lhs->strides[lhs->ndim-1-i];
    }
    for(uint8_t i=lhs->ndim; i < ndim; i++) {
        lstride[ndim-1-i] = lhs->strides[0];        
    }
    for(uint8_t i=0; i < rhs->ndim; i++) {
        rstride[ndim-1-i] = rhs->strides[rhs->ndim-1-i];
    }
    for(uint8_t i=rhs->ndim; i < ndim; i++) {
        rstride[ndim-1-i] = rhs->strides[0];        
    }
    
    mp_float_t lvalue, rvalue, result = 0.0;
    int32_t ilvalue, irvalue, iresult = 0;
    uint8_t float_size = sizeof(mp_float_t);
    void *narray = ndarray->array->items;
    uint8_t *larray = (uint8_t *)lhs->array->items;
    uint8_t *rarray = (uint8_t *)rhs->array->items;
    for(size_t i=0; i < ndarray->len; i++) {
        // we could make this prettier by moving everything into a function,
        // but the function call might be too expensive, especially that it is in the loop...
        // left hand side
        if(lhs->array->typecode == NDARRAY_UINT8) {
            ilvalue = (int32_t)((uint8_t *)larray)[loffset];
        } else if(lhs->array->typecode == NDARRAY_INT8) {
            ilvalue = (int32_t)((int8_t *)larray)[loffset];
        } else if(lhs->array->typecode == NDARRAY_UINT16) {
            ilvalue = (int32_t)((uint16_t *)larray)[loffset*2];
        } else if(lhs->array->typecode == NDARRAY_INT16) {
            ilvalue = (int32_t)((int16_t *)larray)[loffset*2];
        } else {
            lvalue = (mp_float_t)((mp_float_t *)larray)[loffset*float_size];
        }
        // right hand side
        if(rhs->array->typecode == NDARRAY_UINT8) {
            irvalue = (int32_t)((uint8_t *)rarray)[roffset];
        } else if(rhs->array->typecode == NDARRAY_INT8) {
            irvalue = (int32_t)((int8_t *)rarray)[roffset];
        } else if(rhs->array->typecode == NDARRAY_UINT16) {
            irvalue = (int32_t)((uint16_t *)rarray)[roffset*2];
        } else if(lhs->array->typecode == NDARRAY_INT16) {
            irvalue = (int32_t)((int16_t *)rarray)[roffset*2];
        } else {
            rvalue = (mp_float_t)((mp_float_t *)rarray)[roffset*float_size];
        }
        // this is the place, where the actual operations take place
        if(op == MP_BINARY_OP_ADD) {
            result = lvalue + rvalue;
        } else if(op == MP_BINARY_OP_SUBTRACT) {
            result = lvalue - rvalue;
        } else if(op == MP_BINARY_OP_MULTIPLY) {
            result = lvalue * rvalue;
        }
        // cast the result to the proper output type
        if(typecode == NDARRAY_UINT8) {
            ((uint8_t *)narray)[offset] = (uint8_t)result;
        } else if(typecode == NDARRAY_INT8) {
            ((int8_t *)narray)[offset] = (int8_t)result;
        } else if(typecode == NDARRAY_UINT16) {
            ((uint16_t *)narray)[offset] = (uint16_t)result;
        } else if(typecode == NDARRAY_INT16) {
            ((int16_t *)narray)[offset] = (int16_t)result;
        } else {
            ((mp_float_t *)narray)[offset] = result;
        }
        
        
        offset += ndarray->strides[ndim-1];
        coords[ndim-1] += 1;
        for(uint8_t j=ndim-1; j > 0; j--) {
            if(coords[j] == ndarray->shape[j]) {
                offset -= ndarray->shape[j] * ndarray->strides[j];
                offset += ndarray->strides[j-1];
                coords[j] = 0;
                coords[j-1] += 1;
            } else { // coordinates can change only, if the last coordinate changes
                break;
            }        
        }   
    }
    m_del(size_t, coords, ndim);
    m_del(size_t, lcoords, ndim);
    m_del(size_t, rcoords, ndim);
    m_del(int32_t, lstride, ndim);
    m_del(int32_t, rstride, ndim);    
    return ndarray;
}

// Binary operations
mp_obj_t ndarray_binary_op(mp_binary_op_t op, mp_obj_t LHS, mp_obj_t RHS) {
    // First, handle the case, when the operand on the right hand side is a scalar
    ndarray_obj_t *lhs = MP_OBJ_TO_PTR(LHS);
    ndarray_obj_t *rhs;
    if(mp_obj_is_int(RHS) || mp_obj_is_float(RHS)) {
        size_t *shape = m_new(size_t, lhs->ndim*sizeof(size_t));
        for(uint8_t i=0; i < lhs->ndim; i++) {
            shape[i] = 1;
        }
        if(mp_obj_is_int(RHS)) {
            int32_t ivalue = mp_obj_get_int(RHS);
            if((ivalue > 0) && (ivalue < 256)) {
                rhs = ndarray_new_dense_ndarray(lhs->ndim, shape, NDARRAY_UINT8);
            } else if((ivalue > 255) && (ivalue < 65535)) {
                rhs = ndarray_new_dense_ndarray(lhs->ndim, shape, NDARRAY_UINT16);
            } else if((ivalue < 0) && (ivalue > -128)) {
                rhs = ndarray_new_dense_ndarray(lhs->ndim, shape, NDARRAY_INT8);
            } else if((ivalue < -127) && (ivalue > -32767)) {
                rhs = ndarray_new_dense_ndarray(lhs->ndim, shape, NDARRAY_INT16);
            } else { // the integer value clearly does not fit the ulab types, so move on to float
                rhs = ndarray_new_dense_ndarray(lhs->ndim, shape, NDARRAY_FLOAT);
            }
            mp_binary_set_val_array(rhs->array->typecode, rhs->array->items, 0, RHS);
        } else { // we have a float
            rhs = ndarray_new_dense_ndarray(lhs->ndim, shape, NDARRAY_FLOAT);
            mp_binary_set_val_array(rhs->array->typecode, rhs->array->items, 0, RHS);
        }
    } else {
        // the right hand side is an ndarray
        rhs = MP_OBJ_TO_PTR(RHS);
    }
    
    ndarray_obj_t *ndarray = NULL;
    size_t *new_shape = broadcasting(lhs, rhs);
    
    switch(op) {
        case MP_BINARY_OP_MAT_MULTIPLY:
            break;
        
        case MP_BINARY_OP_ADD:
            if(new_shape[0] == 0) {
                mp_raise_ValueError("operands could not be cast together");
            }
            ndarray = ndarray_do_binary_op(lhs, rhs, op);
            // The parameters of RUN_BINARY_LOOP are 
            // typecode of type_out, type_left, type_right, out_array, lhs_array, rhs_array, shape, ndim, operator
            //RUN_BINARY_LOOP(ndarray, NDARRAY_FLOAT, mp_float_t, mp_float_t, mp_float_t, lhs, rhs, new_shape, ndim, operator);
            /* if(lhs->array->typecode == NDARRAY_UINT8) {
                if(rhs->array->typecode == NDARRAY_UINT8) {
                    RUN_BINARY_LOOP(NDARRAY_UINT8, uint8_t, uint8_t, uint8_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_INT8) {
                    RUN_BINARY_LOOP(NDARRAY_INT16, int16_t, uint8_t, int8_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_UINT16) {
                    RUN_BINARY_LOOP(NDARRAY_UINT16, uint16_t, uint8_t, uint16_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_INT16) {
                    RUN_BINARY_LOOP(NDARRAY_INT16, int16_t, uint8_t, int16_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_FLOAT) {
                    RUN_BINARY_LOOP(NDARRAY_FLOAT, mp_float_t, uint8_t, mp_float_t, lhs, rhs, new_shape, ndim, operator);
                }
            } else if(lhs->array->typecode == NDARRAY_INT8) {
                if(rhs->array->typecode == NDARRAY_UINT8) {
                    RUN_BINARY_LOOP(NDARRAY_INT16, int16_t, int8_t, uint8_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_INT8) {
                    RUN_BINARY_LOOP(NDARRAY_INT8, int8_t, int8_t, int8_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_UINT16) {
                    RUN_BINARY_LOOP(NDARRAY_INT16, int16_t, int8_t, uint16_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_INT16) {
                    RUN_BINARY_LOOP(NDARRAY_INT16, int16_t, int8_t, int16_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_FLOAT) {
                    RUN_BINARY_LOOP(NDARRAY_FLOAT, mp_float_t, int8_t, mp_float_t, lhs, rhs, new_shape, ndim, operator);
                }                
            } else if(lhs->array->typecode == NDARRAY_UINT16) {
                if(rhs->array->typecode == NDARRAY_UINT8) {
                    RUN_BINARY_LOOP(NDARRAY_UINT16, uint16_t, uint16_t, uint8_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_INT8) {
                    RUN_BINARY_LOOP(NDARRAY_UINT16, uint16_t, uint16_t, int8_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_UINT16) {
                    RUN_BINARY_LOOP(NDARRAY_UINT16, uint16_t, uint16_t, uint16_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_INT16) {
                    RUN_BINARY_LOOP(NDARRAY_FLOAT, mp_float_t, uint16_t, int16_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_FLOAT) {
                    RUN_BINARY_LOOP(NDARRAY_FLOAT, mp_float_t, uint8_t, mp_float_t, lhs, rhs, new_shape, ndim, operator);
                }
            } else if(lhs->array->typecode == NDARRAY_INT16) {
                if(rhs->array->typecode == NDARRAY_UINT8) {
                    RUN_BINARY_LOOP(NDARRAY_INT16, int16_t, int16_t, uint8_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_INT8) {
                    RUN_BINARY_LOOP(NDARRAY_INT16, int16_t, int16_t, int8_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_UINT16) {
                    RUN_BINARY_LOOP(NDARRAY_FLOAT, mp_float_t, int16_t, uint16_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_INT16) {
                    RUN_BINARY_LOOP(NDARRAY_INT16, int16_t, int16_t, int16_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_FLOAT) {
                    RUN_BINARY_LOOP(NDARRAY_FLOAT, mp_float_t, uint16_t, mp_float_t, lhs, rhs, new_shape, ndim, operator);
                }
            } else if(lhs->array->typecode == NDARRAY_FLOAT) {
                if(rhs->array->typecode == NDARRAY_UINT8) {
                    RUN_BINARY_LOOP(NDARRAY_FLOAT, mp_float_t, mp_float_t, uint8_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_INT8) {
                    RUN_BINARY_LOOP(NDARRAY_FLOAT, mp_float_t, mp_float_t, int8_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_UINT16) {
                    RUN_BINARY_LOOP(NDARRAY_FLOAT, mp_float_t, mp_float_t, uint16_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_INT16) {
                    RUN_BINARY_LOOP(NDARRAY_FLOAT, mp_float_t, mp_float_t, int16_t, lhs, rhs, new_shape, ndim, operator);
                } else if(rhs->array->typecode == NDARRAY_FLOAT) {
                    RUN_BINARY_LOOP(NDARRAY_FLOAT, mp_float_t, mp_float_t, mp_float_t, lhs, rhs, new_shape, ndim, operator);
                } 
            }  else { // this should never happen
                mp_raise_TypeError("wrong input type");
            } */
        break;
        default:
//            m_del(size_t, shape, lhs->ndim*sizeof(size_t));
            return mp_const_none;
        }
    
    
    return MP_OBJ_FROM_PTR(ndarray);
//    return mp_const_none;
}

mp_obj_t ndarray_unary_op(mp_unary_op_t op, mp_obj_t self_in) {
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    ndarray_obj_t *ndarray;

    switch(op) {
        case MP_UNARY_OP_LEN:
            return mp_obj_new_int(self->shape[0]);
            break;

        case MP_UNARY_OP_INVERT:
            if(self->array->typecode == NDARRAY_FLOAT) {
                mp_raise_ValueError("operation is not supported for given type");
            }
            // we can invert the content byte by byte, there is no need to distinguish between different typecodes
            ndarray = ndarray_copy_view(self, self->array->typecode);
            uint8_t *array = (uint8_t *)ndarray->array->items;
            if(self->boolean == NDARRAY_BOOLEAN) {
                for(size_t i=0; i < self->len; i++) array[i] = 1 - array[i];
            }
            else {
                for(size_t i=0; i < self->len; i++) array[i] = ~array[i];
            }
            return MP_OBJ_FROM_PTR(ndarray);
            break;
        
        case MP_UNARY_OP_NEGATIVE:
            if(self->boolean == NDARRAY_BOOLEAN) {
                mp_raise_TypeError("boolean negative '-' is not supported, use the '~' operator instead");
            }
            ndarray = ndarray_copy_view(self, self->array->typecode);
            if(self->array->typecode == NDARRAY_UINT8) {
                uint8_t *array = (uint8_t *)ndarray->array->items;
                for(size_t i=0; i < self->len; i++) array[i] = -array[i];
            } else if(self->array->typecode == NDARRAY_INT8) {
                int8_t *array = (int8_t *)ndarray->array->items;
                for(size_t i=0; i < self->len; i++) array[i] = -array[i];
            } else if(self->array->typecode == NDARRAY_UINT16) {
                uint16_t *array = (uint16_t *)ndarray->array->items;
                for(size_t i=0; i < self->len; i++) array[i] = -array[i];
            } else if(self->array->typecode == NDARRAY_INT16) {
                int16_t *array = (int16_t *)ndarray->array->items;
                for(size_t i=0; i < self->len; i++) array[i] = -array[i];
            } else {
                mp_float_t *array = (mp_float_t *)ndarray->array->items;
                for(size_t i=0; i < self->len; i++) array[i] = -array[i];
            }
            return MP_OBJ_FROM_PTR(ndarray);
            break;

        case MP_UNARY_OP_POSITIVE:
            return MP_OBJ_FROM_PTR(ndarray_copy_view(self, self->array->typecode));

        case MP_UNARY_OP_ABS:
            if((self->array->typecode == NDARRAY_UINT8) || (self->array->typecode == NDARRAY_UINT16)) {
                return MP_OBJ_FROM_PTR(ndarray_copy_view(self, self->array->typecode));
            }
            ndarray = ndarray_copy_view(self, self->array->typecode);
            if(self->array->typecode == NDARRAY_INT8) {
                int8_t *array = (int8_t *)ndarray->array->items;
                for(size_t i=0; i < self->len; i++) {
                    if(array[i] < 0) array[i] = -array[i];
                }
            } else if(self->array->typecode == NDARRAY_INT16) {
                int16_t *array = (int16_t *)ndarray->array->items;
                for(size_t i=0; i < self->len; i++) {
                    if(array[i] < 0) array[i] = -array[i];
                }
            } else {
                mp_float_t *array = (mp_float_t *)ndarray->array->items;
                for(size_t i=0; i < self->array->len; i++) {
                    if(array[i] < 0) array[i] = -array[i];
                }
            }
            return MP_OBJ_FROM_PTR(ndarray);
        break;

        default: return MP_OBJ_NULL; // operator not supported
    }
}

written 44617 bytes to ndarray.c


# Vectorise

## vectorise.h

In [46]:
%%ccode vectorise.h

#ifndef _VECTORISE_
#define _VECTORISE_

#include "ndarray.h"

mp_obj_t vectorise_acos(mp_obj_t );
mp_obj_t vectorise_acosh(mp_obj_t );
mp_obj_t vectorise_asin(mp_obj_t );
mp_obj_t vectorise_asinh(mp_obj_t );
mp_obj_t vectorise_atan(mp_obj_t );
mp_obj_t vectorise_atanh(mp_obj_t );
mp_obj_t vectorise_ceil(mp_obj_t );
mp_obj_t vectorise_cos(mp_obj_t );
mp_obj_t vectorise_erf(mp_obj_t );
mp_obj_t vectorise_erfc(mp_obj_t );
mp_obj_t vectorise_exp(mp_obj_t );
mp_obj_t vectorise_expm1(mp_obj_t );
mp_obj_t vectorise_floor(mp_obj_t );
mp_obj_t vectorise_gamma(mp_obj_t );
mp_obj_t vectorise_lgamma(mp_obj_t );
mp_obj_t vectorise_log(mp_obj_t );
mp_obj_t vectorise_log10(mp_obj_t );
mp_obj_t vectorise_log2(mp_obj_t );
mp_obj_t vectorise_sin(mp_obj_t );
mp_obj_t vectorise_sinh(mp_obj_t );
mp_obj_t vectorise_sqrt(mp_obj_t );
mp_obj_t vectorise_tan(mp_obj_t );
mp_obj_t vectorise_tanh(mp_obj_t );

#define ITERATE_VECTOR(type, source, out) do {\
    type *input = (type *)(source)->array->items;\
    for(size_t i=0; i < (source)->array->len; i++) {\
        *(out)++ = f(*input++);\
    }\
} while(0)

#define MATH_FUN_1(py_name, c_name) \
    mp_obj_t vectorise_ ## py_name(mp_obj_t x_obj) { \
        return vectorise_generic_vector(x_obj, MICROPY_FLOAT_C_FUN(c_name)); \
    }
    
#endif

written 1470 bytes to vectorise.h


## vectorise.c

In [42]:
%%ccode vectorise.c

#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include "py/runtime.h"
#include "py/binary.h"
#include "py/obj.h"
#include "py/objarray.h"
#include "vectorise.h"

#ifndef MP_PI
#define MP_PI MICROPY_FLOAT_CONST(3.14159265358979323846)
#endif
    
mp_obj_t vectorise_generic_vector(mp_obj_t o_in, mp_float_t (*f)(mp_float_t)) {
    // Return a single value, if o_in is not iterable
    if(mp_obj_is_float(o_in) || mp_obj_is_integer(o_in)) {
        return mp_obj_new_float(f(mp_obj_get_float(o_in)));
    }
    if(MP_OBJ_IS_TYPE(o_in, &ulab_ndarray_type)) {
        ndarray_obj_t *source = MP_OBJ_TO_PTR(o_in);
        ndarray_obj_t *ndarray;
        if(source->len == source->array->len) { // this is a dense array
            ndarray = ndarray_new_dense_ndarray(source->ndim, source->shape, NDARRAY_FLOAT);
            mp_float_t *array = (mp_float_t *)ndarray->array->items;
            if(source->array->typecode == NDARRAY_UINT8) {
                ITERATE_VECTOR(uint8_t, source, array);
            } else if(source->array->typecode == NDARRAY_INT8) {
                ITERATE_VECTOR(int8_t, source, array);
            } else if(source->array->typecode == NDARRAY_UINT16) {
                ITERATE_VECTOR(uint16_t, source, array);
            } else if(source->array->typecode == NDARRAY_INT16) {
                ITERATE_VECTOR(int16_t, source, array);
            } else {
                ITERATE_VECTOR(mp_float_t, source, array);
            } 
        } else {
            ndarray = ndarray_copy_view(source, NDARRAY_FLOAT);
            mp_float_t *dataout = (mp_float_t *)ndarray->array->items;
            ITERATE_VECTOR(mp_float_t, ndarray, dataout);
        }
        return MP_OBJ_FROM_PTR(ndarray);
    } else if(MP_OBJ_IS_TYPE(o_in, &mp_type_tuple) || MP_OBJ_IS_TYPE(o_in, &mp_type_list) || 
        MP_OBJ_IS_TYPE(o_in, &mp_type_range)) { // i.e., the input is a generic, one-dimensional iterable
            mp_obj_array_t *o = MP_OBJ_TO_PTR(o_in);
            ndarray_obj_t *out = ndarray_new_linear_array(o->len, NDARRAY_FLOAT);
            mp_float_t *dataout = (mp_float_t *)out->array->items;
            
            mp_obj_iter_buf_t iter_buf;
            mp_obj_t item, iterable = mp_getiter(o_in, &iter_buf);
            mp_float_t x;
            while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
                x = mp_obj_get_float(item);
                *dataout++ = f(x);
            }
        return MP_OBJ_FROM_PTR(out);
    }
    return mp_const_none;
}

MATH_FUN_1(acos, acos);
MATH_FUN_1(acosh, acosh);
MATH_FUN_1(asin, asin);
MATH_FUN_1(asinh, asinh);
MATH_FUN_1(atan, atan);
MATH_FUN_1(atanh, atanh);
MATH_FUN_1(ceil, ceil);
MATH_FUN_1(cos, cos);
MATH_FUN_1(erf, erf);
MATH_FUN_1(erfc, erfc);
MATH_FUN_1(exp, exp);
MATH_FUN_1(expm1, expm1);
MATH_FUN_1(floor, floor);
MATH_FUN_1(gamma, tgamma);
MATH_FUN_1(lgamma, lgamma);
MATH_FUN_1(log, log);
MATH_FUN_1(log10, log10);
MATH_FUN_1(log2, log2);
MATH_FUN_1(sin, sin);
MATH_FUN_1(sinh, sinh);
MATH_FUN_1(sqrt, sqrt);
MATH_FUN_1(tan, tan);
MATH_FUN_1(tanh, tanh);

written 3254 bytes to vectorise.c


# Linalg

## linalg.h

In [62]:
%%ccode linalg.h

#ifndef _LINALG_
#define _LINALG_

#include "ndarray.h"

#if MICROPY_FLOAT_IMPL == MICROPY_FLOAT_IMPL_FLOAT
#define epsilon        1.2e-7
#elif MICROPY_FLOAT_IMPL == MICROPY_FLOAT_IMPL_DOUBLE
#define epsilon        2.3e-16
#endif

#define JACOBI_MAX     20

bool linalg_invert_matrix(mp_float_t *, size_t );
mp_obj_t linalg_inv(mp_obj_t );
mp_obj_t linalg_dot(mp_obj_t , mp_obj_t );
mp_obj_t linalg_zeros(size_t , const mp_obj_t *, mp_map_t *);
mp_obj_t linalg_ones(size_t , const mp_obj_t *, mp_map_t *);
mp_obj_t linalg_eye(size_t , const mp_obj_t *, mp_map_t *);

mp_obj_t linalg_det(mp_obj_t );
mp_obj_t linalg_eig(mp_obj_t );

#endif

written 819 bytes to linalg.h


## linalg.c

In [48]:
%%ccode linalg.c

#include <stdlib.h>
#include <string.h>
#include <math.h>
#include "py/obj.h"
#include "py/runtime.h"
#include "py/misc.h"
#include "linalg.h"

bool linalg_invert_matrix(mp_float_t *data, size_t N) {
    // returns true, of the inversion was successful, 
    // false, if the matrix is singular
    
    // initially, this is the unit matrix: the contents of this matrix is what 
    // will be returned after all the transformations
    mp_float_t *unit = m_new(mp_float_t, N*N);

    mp_float_t elem = 1.0;
    // initialise the unit matrix
    memset(unit, 0, sizeof(mp_float_t)*N*N);
    for(size_t m=0; m < N; m++) {
        memcpy(&unit[m*(N+1)], &elem, sizeof(mp_float_t));
    }
    for(size_t m=0; m < N; m++){
        // this could be faster with ((c < epsilon) && (c > -epsilon))
        if(MICROPY_FLOAT_C_FUN(fabs)(data[m*(N+1)]) < epsilon) {
            m_del(mp_float_t, unit, N*N);
            return false;
        }
        for(size_t n=0; n < N; n++){
            if(m != n){
                elem = data[N*n+m] / data[m*(N+1)];
                for(size_t k=0; k < N; k++){
                    data[N*n+k] -= elem * data[N*m+k];
                    unit[N*n+k] -= elem * unit[N*m+k];
                }
            }
        }
    }
    for(size_t m=0; m < N; m++){ 
        elem = data[m*(N+1)];
        for(size_t n=0; n < N; n++){
            data[N*m+n] /= elem;
            unit[N*m+n] /= elem;
        }
    }
    memcpy(data, unit, sizeof(mp_float_t)*N*N);
    m_del(mp_float_t, unit, N*N);
    return true;
}

mp_obj_t linalg_inv(mp_obj_t o_in) {
    if(!MP_OBJ_IS_TYPE(o_in, &ulab_ndarray_type)) {
        mp_raise_TypeError("only ndarray objects can be inverted");
    }
    ndarray_obj_t *ndarray = MP_OBJ_TO_PTR(o_in);
    if(ndarray->ndim != 2) {
        mp_raise_ValueError("only two-dimensional tensors can be inverted");
    }
    if(ndarray->shape[0] != ndarray->shape[1]) {
        mp_raise_ValueError("only square matrices can be inverted");
    }
    size_t *shape = m_new(size_t, 2);
    shape[0] = shape[1] = ndarray->shape[0];
    ndarray_obj_t *inverted = ndarray_new_dense_ndarray(2, shape, NDARRAY_FLOAT);
    mp_float_t *data = (mp_float_t *)inverted->array->items;
    mp_obj_t elem;
    for(size_t m=0; m < ndarray->shape[0]; m++) { // rows first
        for(size_t n=0; n < ndarray->shape[1]; n++) { // columns next
            // this could, perhaps, be done in single line... 
            // On the other hand, we probably spend little time here
            elem = mp_binary_get_val_array(ndarray->array->typecode, ndarray->array->items, m*ndarray->shape[1]+n);
            data[m*ndarray->shape[1]+n] = (mp_float_t)mp_obj_get_float(elem);
        }
    }
    
    if(!linalg_invert_matrix(data, ndarray->shape[0])) {
        // TODO: I am not sure this is needed here. Otherwise, how should we free up the unused RAM of inverted?
        m_del(mp_float_t, inverted->array->items, ndarray->shape[0]*ndarray->shape[1]);
        mp_raise_ValueError("input matrix is singular");
    }
    return MP_OBJ_FROM_PTR(inverted);
}

mp_obj_t linalg_dot(mp_obj_t _m1, mp_obj_t _m2) {
    return mp_const_none;
    /*
    // TODO: should the results be upcast?
    ndarray_obj_t *m1 = MP_OBJ_TO_PTR(_m1);
    ndarray_obj_t *m2 = MP_OBJ_TO_PTR(_m2);    
    if(m1->n != m2->m) {
        mp_raise_ValueError("matrix dimensions do not match");
    }
    // TODO: numpy uses upcasting here
    ndarray_obj_t *out = create_new_ndarray(m1->m, m2->n, NDARRAY_FLOAT);
    mp_float_t *outdata = (mp_float_t *)out->array->items;
    mp_float_t sum, v1, v2;
    for(size_t i=0; i < m1->m; i++) { // rows of m1
        for(size_t j=0; j < m2->n; j++) { // columns of m2
            sum = 0.0;
            for(size_t k=0; k < m2->m; k++) {
                // (i, k) * (k, j)
                v1 = ndarray_get_float_value(m1->array->items, m1->array->typecode, i*m1->n+k);
                v2 = ndarray_get_float_value(m2->array->items, m2->array->typecode, k*m2->n+j);
                sum += v1 * v2;
            }
            outdata[i*m1->m+j] = sum;
        }
    }
    return MP_OBJ_FROM_PTR(out); */
} 

mp_obj_t linalg_zeros_ones(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args, uint8_t kind) {
    static const mp_arg_t allowed_args[] = {
        { MP_QSTR_, MP_ARG_REQUIRED | MP_ARG_OBJ, {.u_obj = MP_OBJ_NULL} } ,
        { MP_QSTR_dtype, MP_ARG_KW_ONLY | MP_ARG_INT, {.u_int = NDARRAY_FLOAT} },
    };

    mp_arg_val_t args[MP_ARRAY_SIZE(allowed_args)];
    mp_arg_parse_all(n_args, pos_args, kw_args, MP_ARRAY_SIZE(allowed_args), allowed_args, args);
    
    uint8_t dtype = args[1].u_int;
    if(!mp_obj_is_int(args[0].u_obj) && !mp_obj_is_type(args[0].u_obj, &mp_type_tuple)) {
        mp_raise_TypeError("input argument must be an integer or a tuple");
    }
    ndarray_obj_t *ndarray = NULL;
    if(mp_obj_is_int(args[0].u_obj)) {
        size_t n = mp_obj_get_int(args[0].u_obj);
        size_t *shape = m_new(size_t, 1);
        shape[0] = n;
        ndarray = ndarray_new_dense_ndarray(1, shape, dtype);
    } else if(mp_obj_is_type(args[0].u_obj, &mp_type_tuple)) {
        mp_obj_tuple_t *tuple = MP_OBJ_TO_PTR(args[0].u_obj);
        size_t *shape = m_new(size_t, tuple->len);
        for(uint8_t i=0; i < tuple->len; i++) {
            shape[i] = mp_obj_get_int(tuple->items[i]);
        }
        ndarray = ndarray_new_dense_ndarray(tuple->len, shape, dtype);
    }
    if(kind == 1) {
        mp_obj_t one = mp_obj_new_int(1);
        for(size_t i=0; i < ndarray->array->len; i++) {
            mp_binary_set_val_array(dtype, ndarray->array->items, i, one);
        }
    }
    return MP_OBJ_FROM_PTR(ndarray);
}

mp_obj_t linalg_zeros(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    return linalg_zeros_ones(n_args, pos_args, kw_args, 0);
}

mp_obj_t linalg_ones(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    return linalg_zeros_ones(n_args, pos_args, kw_args, 1);
}

mp_obj_t linalg_eye(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    // TODO: this is a bit more problematic in higher dimensions
    return mp_const_none;
    /*
    static const mp_arg_t allowed_args[] = {
        { MP_QSTR_, MP_ARG_REQUIRED | MP_ARG_INT, {.u_int = 0} },
        { MP_QSTR_M, MP_ARG_KW_ONLY | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_none_obj) } },
        { MP_QSTR_k, MP_ARG_KW_ONLY | MP_ARG_INT, {.u_int = 0} },        
        { MP_QSTR_dtype, MP_ARG_KW_ONLY | MP_ARG_INT, {.u_int = NDARRAY_FLOAT} },
    };

    mp_arg_val_t args[MP_ARRAY_SIZE(allowed_args)];
    mp_arg_parse_all(n_args, pos_args, kw_args, MP_ARRAY_SIZE(allowed_args), allowed_args, args);

    size_t n = args[0].u_int, m;
    int16_t k = args[2].u_int;
    uint8_t dtype = args[3].u_int;
    if(args[1].u_rom_obj == mp_const_none) {
        m = n;
    } else {
        m = mp_obj_get_int(args[1].u_rom_obj);
    }
    
    ndarray_obj_t *ndarray = create_new_ndarray(m, n, dtype);
    mp_obj_t one = mp_obj_new_int(1);
    size_t i = 0;
    if((k >= 0) && (k < n)) {
        while(k < n) {
            mp_binary_set_val_array(dtype, ndarray->array->items, i*n+k, one);
            k++;
            i++;
        }
    } else if((k < 0) && (-k < m)) {
        k = -k;
        i = 0;
        while(k < m) {
            mp_binary_set_val_array(dtype, ndarray->array->items, k*n+i, one);
            k++;
            i++;
        }
    }
    return MP_OBJ_FROM_PTR(ndarray); */
}

mp_obj_t linalg_det(mp_obj_t oin) {
    if(!mp_obj_is_type(oin, &ulab_ndarray_type)) {
        mp_raise_TypeError("function defined for ndarrays only");
    }
    ndarray_obj_t *ndarray = MP_OBJ_TO_PTR(oin);
    if(ndarray->ndim != 2) {
        mp_raise_ValueError("only two-dimensional tensors can be inverted");
    }
    if(ndarray->shape[0] != ndarray->shape[1]) {
        mp_raise_ValueError("only square matrices can be inverted");
    }
    
    mp_float_t *tmp = m_new(mp_float_t, ndarray->shape[0]*ndarray->shape[1]);
    // TODO: this won't work for sliced arrays
    for(size_t i=0; i < ndarray->len; i++){
        tmp[i] = ndarray_get_float_value(ndarray->array->items, ndarray->array->typecode, i);
    }
    mp_float_t c;
    for(size_t m=0; m < ndarray->shape[0]-1; m++){
        if(MICROPY_FLOAT_C_FUN(fabs)(tmp[m*(ndarray->shape[1]+1)]) < epsilon) {
            m_del(mp_float_t, tmp, ndarray->shape[0]*ndarray->shape[1]);
            return mp_obj_new_float(0.0);
        }
        for(size_t n=0; n < ndarray->shape[1]; n++){
            if(m != n) {
                c = tmp[ndarray->shape[0]*n+m] / tmp[m*(ndarray->shape[1]+1)];
                for(size_t k=0; k < ndarray->shape[1]; k++){
                    tmp[ndarray->shape[1]*n+k] -= c * tmp[ndarray->shape[1]*m+k];
                }
            }
        }
    }
    mp_float_t det = 1.0;
                            
    for(size_t m=0; m < ndarray->shape[0]; m++){ 
        det *= tmp[m*(ndarray->shape[1]+1)];
    }
    m_del(mp_float_t, tmp, ndarray->shape[0]*ndarray->shape[1]);
    return mp_obj_new_float(det);
}

mp_obj_t linalg_eig(mp_obj_t oin) {
    if(!mp_obj_is_type(oin, &ulab_ndarray_type)) {
        mp_raise_TypeError("function defined for ndarrays only");
    }
    ndarray_obj_t *ndarray = MP_OBJ_TO_PTR(oin);
    if(ndarray->ndim != 2) {
        mp_raise_ValueError("only two-dimensional tensors can be inverted");
    }
    if(ndarray->shape[0] != ndarray->shape[1]) {
        mp_raise_ValueError("only square matrices can be inverted");
    }
    mp_float_t *array = m_new(mp_float_t, ndarray->len);
    // TODO: this won't work for sliced arrays
    for(size_t i=0; i < ndarray->len; i++) {
        array[i] = ndarray_get_float_value(ndarray->array->items, ndarray->array->typecode, i);
    }
    // make sure the matrix is symmetric
    for(size_t m=0; m < ndarray->shape[0]; m++) {
        for(size_t n=m+1; n < ndarray->shape[1]; n++) {
            // compare entry (m, n) to (n, m)
            // TODO: this must probably be scaled!
            if(epsilon < MICROPY_FLOAT_C_FUN(fabs)(array[m*ndarray->shape[0] + n] - array[n*ndarray->shape[0] + m])) {
                mp_raise_ValueError("input matrix is asymmetric");
            }
        }
    }
    
    // if we got this far, then the matrix will be symmetric
    size_t *shape = m_new(size_t, 2);
    shape[0] = shape[1] = ndarray->shape[0];
    ndarray_obj_t *eigenvectors = ndarray_new_dense_ndarray(2, shape, NDARRAY_FLOAT);
    mp_float_t *eigvectors = (mp_float_t *)eigenvectors->array->items;
    // start out with the unit matrix
    for(size_t m=0; m < ndarray->shape[0]; m++) {
        eigvectors[m*(ndarray->shape[1]+1)] = 1.0;
    }
    mp_float_t largest, w, t, c, s, tau, aMk, aNk, vm, vn;
    size_t M, N;
    size_t iterations = JACOBI_MAX*ndarray->shape[0]*ndarray->shape[0];
    do {
        iterations--;
        // find the pivot here
        M = 0;
        N = 0;
        largest = 0.0;
        for(size_t m=0; m < ndarray->shape[0]-1; m++) { // -1: no need to inspect last row
            for(size_t n=m+1; n < ndarray->shape[0]; n++) {
                w = MICROPY_FLOAT_C_FUN(fabs)(array[m*ndarray->shape[0] + n]);
                if((largest < w) && (epsilon < w)) {
                    M = m;
                    N = n;
                    largest = w;
                }
            }
        }
        if(M+N == 0) { // all entries are smaller than epsilon, there is not much we can do...
            break;
        }
        // at this point, we have the pivot, and it is the entry (M, N)
        // now we have to find the rotation angle
        w = (array[N*ndarray->shape[0] + N] - array[M*ndarray->shape[0] + M]) / (2.0*array[M*ndarray->shape[0] + N]);
        // The following if/else chooses the smaller absolute value for the tangent 
        // of the rotation angle. Going with the smaller should be numerically stabler.
        if(w > 0) {
            t = MICROPY_FLOAT_C_FUN(sqrt)(w*w + 1.0) - w;
        } else {
            t = -1.0*(MICROPY_FLOAT_C_FUN(sqrt)(w*w + 1.0) + w);
        }
        s = t / MICROPY_FLOAT_C_FUN(sqrt)(t*t + 1.0); // the sine of the rotation angle
        c = 1.0 / MICROPY_FLOAT_C_FUN(sqrt)(t*t + 1.0); // the cosine of the rotation angle
        tau = (1.0-c)/s; // this is equal to the tangent of the half of the rotation angle
        
        // at this point, we have the rotation angles, so we can transform the matrix
        // first the two diagonal elements
        // a(M, M) = a(M, M) - t*a(M, N)
        array[M*ndarray->shape[0] + M] = array[M*ndarray->shape[0] + M] - t * array[M*ndarray->shape[0] + N];
        // a(N, N) = a(N, N) + t*a(M, N)
        array[N*ndarray->shape[0] + N] = array[N*ndarray->shape[0] + N] + t * array[M*ndarray->shape[0] + N];
        // after the rotation, the a(M, N), and a(N, M) entries should become zero
        array[M*ndarray->shape[0] + N] = array[N*ndarray->shape[0] + M] = 0.0;
        // then all other elements in the column
        for(size_t k=0; k < ndarray->shape[0]; k++) {
            if((k == M) || (k == N)) {
                continue;
            }
            aMk = array[M*ndarray->shape[0] + k];
            aNk = array[N*ndarray->shape[0] + k];
            // a(M, k) = a(M, k) - s*(a(N, k) + tau*a(M, k))
            array[M*ndarray->shape[0] + k] -= s*(aNk + tau*aMk);
            // a(N, k) = a(N, k) + s*(a(M, k) - tau*a(N, k))
            array[N*ndarray->shape[0] + k] += s*(aMk - tau*aNk);
            // a(k, M) = a(M, k)
            array[k*ndarray->shape[0] + M] = array[M*ndarray->shape[0] + k];
            // a(k, N) = a(N, k)
            array[k*ndarray->shape[0] + N] = array[N*ndarray->shape[0] + k];
        }
        // now we have to update the eigenvectors
        // the rotation matrix, R, multiplies from the right
        // R is the unit matrix, except for the 
        // R(M,M) = R(N, N) = c
        // R(N, M) = s
        // (M, N) = -s
        // entries. This means that only the Mth, and Nth columns will change
        for(size_t m=0; m < ndarray->shape[0]; m++) {
            vm = eigvectors[m*ndarray->shape[0]+M];
            vn = eigvectors[m*ndarray->shape[0]+N];
            // the new value of eigvectors(m, M)
            eigvectors[m*ndarray->shape[0]+M] = c * vm - s * vn;
            // the new value of eigvectors(m, N)
            eigvectors[m*ndarray->shape[0]+N] = s * vm + c * vn;
        }
    } while(iterations > 0);
    
    if(iterations == 0) { 
        // the computation did not converge; numpy raises LinAlgError
        m_del(mp_float_t, array, ndarray->len);
        mp_raise_ValueError("iterations did not converge");
    }
    size_t *eigen_shape = m_new(size_t, 1);
    eigen_shape[0] = ndarray->shape[0];
    ndarray_obj_t *eigenvalues = ndarray_new_dense_ndarray(1, eigen_shape, NDARRAY_FLOAT);
    mp_float_t *eigvalues = (mp_float_t *)eigenvalues->array->items;
    for(size_t i=0; i < ndarray->shape[0]; i++) {
        eigvalues[i] = array[i*(ndarray->shape[0]+1)];
    }
    m_del(mp_float_t, array, ndarray->len);
    
    mp_obj_tuple_t *tuple = MP_OBJ_TO_PTR(mp_obj_new_tuple(2, NULL));
    tuple->items[0] = MP_OBJ_FROM_PTR(eigenvalues);
    tuple->items[1] = MP_OBJ_FROM_PTR(eigenvectors);
    return tuple;
    return MP_OBJ_FROM_PTR(eigenvalues);
}

written 15460 bytes to linalg.c


# Polynomial

## poly.h

In [104]:
%%ccode poly.h

#ifndef _POLY_
#define _POLY_

mp_obj_t poly_polyval(mp_obj_t , mp_obj_t );
mp_obj_t poly_polyfit(size_t  , const mp_obj_t *);

#endif

written 315 bytes to poly.h


## poly.c

In [117]:
%%ccode poly.c

#include "py/obj.h"
#include "py/runtime.h"
#include "py/objarray.h"
#include "ndarray.h"
#include "linalg.h"
#include "poly.h"

void fill_array_iterable(mp_float_t *array, mp_obj_t oin) {
    mp_obj_iter_buf_t buf;
    mp_obj_t item, iterable = mp_getiter(oin, &buf);
    while((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
        *array++ = mp_obj_get_float(item);
    }
}

mp_obj_t poly_polyval(mp_obj_t o_p, mp_obj_t o_x) {
    // we always return floats: polynomials are going to be of type float, except, 
    // when both the coefficients and the independent variable are integers; 
    uint8_t plen = mp_obj_get_int(mp_obj_len_maybe(o_p));
    mp_float_t *p = m_new(mp_float_t, plen);
    fill_array_iterable(p, o_p);
    ndarray_obj_t *ndarray;
    mp_float_t *array;
    if(MP_OBJ_IS_TYPE(o_x, &ulab_ndarray_type)) {
        ndarray_obj_t *input = MP_OBJ_TO_PTR(o_x);
        ndarray = ndarray_copy_view(input, NDARRAY_FLOAT);
        array = (mp_float_t *)ndarray->array->items;
    } else { // at this point, we should have a 1-D iterable
        size_t len = mp_obj_get_int(mp_obj_len_maybe(o_x));
        ndarray = ndarray_new_linear_array(len, NDARRAY_FLOAT);
        array = (mp_float_t *)ndarray->array->items;
        fill_array_iterable(array, o_x);
    }
    mp_float_t x, y;
    for(size_t i=0; i < ndarray->len; i++) {
        x = array[i];
        y = p[0];
        for(uint8_t j=0; j < plen-1; j++) {
            y *= x;
            y += p[j+1];
        }
        array[i] = y;
    }
    m_del(mp_float_t, p, plen);
    return MP_OBJ_FROM_PTR(ndarray);
}

mp_obj_t poly_polyfit(size_t n_args, const mp_obj_t *args) {
    if(n_args != 3) {
        mp_raise_ValueError("number of arguments must be 3");
    }
    if(!MP_OBJ_IS_TYPE(args[0], &ulab_ndarray_type) && !MP_OBJ_IS_TYPE(args[0], &mp_type_tuple) &&
      !MP_OBJ_IS_TYPE(args[0], &mp_type_list) && !MP_OBJ_IS_TYPE(args[0], &mp_type_range) &&
      !MP_OBJ_IS_TYPE(args[1], &ulab_ndarray_type) && !MP_OBJ_IS_TYPE(args[1], &mp_type_tuple) &&
      !MP_OBJ_IS_TYPE(args[1], &mp_type_list) && !MP_OBJ_IS_TYPE(args[1], &mp_type_range)) {
        mp_raise_ValueError("input data must be a 1D iterable");
    }
    uint16_t lenx, leny;
    uint8_t deg;
    mp_float_t *x, *XT, *y, *prod;

    lenx = (uint16_t)mp_obj_get_int(mp_obj_len_maybe(args[0]));
    leny = (uint16_t)mp_obj_get_int(mp_obj_len_maybe(args[1]));
    if(lenx != leny) {
        mp_raise_ValueError("input vectors must be of equal length");
    }
    deg = (uint8_t)mp_obj_get_int(args[2]);
    if(leny < deg) {
        mp_raise_ValueError("more degrees of freedom than data points");
    }
    x = m_new(mp_float_t, lenx);
    fill_array_iterable(x, args[0]);
    y = m_new(mp_float_t, leny);
    fill_array_iterable(y, args[1]);
    
    // one could probably express X as a function of XT, 
    // and thereby save RAM, because X is used only in the product
    XT = m_new(mp_float_t, (deg+1)*leny); // XT is a matrix of shape (deg+1, len) (rows, columns)
    for(uint8_t i=0; i < leny; i++) { // column index
        XT[i+0*lenx] = 1.0; // top row
        for(uint8_t j=1; j < deg+1; j++) { // row index
            XT[i+j*leny] = XT[i+(j-1)*leny]*x[i];
        }
    }
    
    prod = m_new(mp_float_t, (deg+1)*(deg+1)); // the product matrix is of shape (deg+1, deg+1)
    mp_float_t sum;
    for(uint16_t i=0; i < deg+1; i++) { // column index
        for(uint16_t j=0; j < deg+1; j++) { // row index
            sum = 0.0;
            for(size_t k=0; k < lenx; k++) {
                // (j, k) * (k, i) 
                // Note that the second matrix is simply the transpose of the first: 
                // X(k, i) = XT(i, k) = XT[k*lenx+i]
                sum += XT[j*lenx+k]*XT[i*lenx+k]; // X[k*(deg+1)+i];
            }
            prod[j*(deg+1)+i] = sum;
        }
    }
    if(!linalg_invert_matrix(prod, deg+1)) {
        // Although X was a Vandermonde matrix, whose inverse is guaranteed to exist, 
        // we bail out here, if prod couldn't be inverted: if the values in x are not all 
        // distinct, prod is singular
        m_del(mp_float_t, XT, (deg+1)*lenx);
        m_del(mp_float_t, x, lenx);
        m_del(mp_float_t, y, lenx);
        m_del(mp_float_t, prod, (deg+1)*(deg+1));
        mp_raise_ValueError("could not invert Vandermonde matrix");
    } 
    // at this point, we have the inverse of X^T * X
    // y is a column vector; x is free now, we can use it for storing intermediate values
    for(uint16_t i=0; i < deg+1; i++) { // row index
        sum = 0.0;
        for(uint16_t j=0; j < lenx; j++) { // column index
            sum += XT[i*lenx+j]*y[j];
        }
        x[i] = sum;
    }
    // XT is no longer needed
    m_del(mp_float_t, XT, (deg+1)*leny);
    
    ndarray_obj_t *beta = ndarray_new_linear_array(deg+1, NDARRAY_FLOAT);
    mp_float_t *betav = (mp_float_t *)beta->array->items;
    // x[0..(deg+1)] contains now the product X^T * y; we can get rid of y
    m_del(float, y, leny);
    
    // now, we calculate beta, i.e., we apply prod = (X^T * X)^(-1) on x = X^T * y; x is a column vector now
    for(uint16_t i=0; i < deg+1; i++) {
        sum = 0.0;
        for(uint16_t j=0; j < deg+1; j++) {
            sum += prod[i*(deg+1)+j]*x[j];
        }
        betav[i] = sum;
    }
    m_del(mp_float_t, x, lenx);
    m_del(mp_float_t, prod, (deg+1)*(deg+1));
    for(uint8_t i=0; i < (deg+1)/2; i++) {
        // We have to reverse the array, for the leading coefficient comes first. 
        SWAP(mp_float_t, betav[i], betav[deg-i]);
    }
    return MP_OBJ_FROM_PTR(beta);
}

written 5781 bytes to poly.c


# Numerical

## numerical.h

In [456]:
%%ccode numerical.h

#ifndef _NUMERICAL_
#define _NUMERICAL_

#include "ndarray.h"

mp_obj_t numerical_linspace(size_t , const mp_obj_t *, mp_map_t *);

mp_obj_t numerical_sum(size_t , const mp_obj_t *, mp_map_t *);
mp_obj_t numerical_mean(size_t , const mp_obj_t *, mp_map_t *);
mp_obj_t numerical_std(size_t , const mp_obj_t *, mp_map_t *);

#define CALCULATE_SUM(ndarray, type, farray, shape_ax, index, stride, offset, optype) do {\
    type *array = (type *)(ndarray)->array->items;\
    (farray)[(index)] = 0.0;\
    for(size_t j=0; j < (shape_ax); j++, (offset) += (stride)) {\
        (farray)[(index)] += array[(offset)];\
    }\
} while(0)

// TODO: this can be done without the NDARRAY_INDEX_FROM_FLAT macro
// Welford algorithm for the standard deviation
#define CALCULATE_FLAT_SUM_STD(ndarray, type, value, shape_strides, optype) do {\
    type *array = (type *)(ndarray)->array->items;\
    (value) = 0.0;\
    mp_float_t m = 0.0, mtmp;\
    size_t index, nindex;\
    for(size_t j=0; j < (ndarray)->len; j++) {\
        NDARRAY_INDEX_FROM_FLAT((ndarray), (shape_strides), j, index, nindex);\
        if((optype) == NUMERICAL_STD) {\
            mtmp = m;\
            m = mtmp + (array[nindex] - mtmp) / (j+1);\
            (value) += (array[nindex] - mtmp) * (array[nindex] - m);\
        } else {\
            (value) += array[nindex];\
        }\
    }\
} while(0)

// we calculate the standard deviation in two passes, in order to avoid negative values through truncation errors
// We could do in a single pass, if we resorted to the Welford algorithm above
#define CALCULATE_STD(ndarray, type, sq_sum, shape_ax, stride, offset) do {\
    type *array = (type *)(ndarray)->array->items;\
    mp_float_t x, ave = 0.0;\
    (sq_sum) = 0.0;\
    size_t j, _offset = (offset);\
    for(j=0; j < (shape_ax); j++, _offset += (stride)) {\
        ave += array[_offset];\
    }\
    ave /= j;\
    for(j=0; j < (shape_ax); j++, (offset) += (stride)) {\
        x = array[(offset)] - ave;\
        (sq_sum) += x * x;\
    }\
} while(0)

#endif

written 2211 bytes to numerical.h


## numerical.c

In [203]:
%%ccode numerical.c

#include <math.h>
#include <stdlib.h>
#include <string.h>
#include "py/obj.h"
#include "py/objint.h"
#include "py/runtime.h"
#include "py/builtin.h"
#include "py/misc.h"
#include "numerical.h"

enum NUMERICAL_FUNCTION_TYPE {
    NUMERICAL_MIN,
    NUMERICAL_MAX,
    NUMERICAL_ARGMIN,
    NUMERICAL_ARGMAX,
    NUMERICAL_SUM,
    NUMERICAL_MEAN,
    NUMERICAL_STD,
};

// creates the shape and strides arrays of the contracted ndarray
ndarray_header_obj_t contracted_shape_strides(ndarray_obj_t *ndarray, int8_t axis) {
    if(axis < 0) axis += ndarray->ndim;
    if((axis > ndarray->ndim-1) || (axis < 0)) {
        mp_raise_ValueError("tuple index out of range");
    }
    size_t *shape = m_new(size_t, ndarray->ndim-1);
    int32_t *strides = m_new(int32_t, ndarray->ndim-1);
    for(size_t i=0, j=0; i < ndarray->ndim; i++) {
        if(axis != i) {
            shape[j] = ndarray->shape[j];
            j++;
        }
    }
    int32_t stride = 1;
    for(size_t i=0; i < ndarray->ndim-1; i++) {
        strides[ndarray->ndim-2-i] = stride;
        stride *= shape[ndarray->ndim-2-i];
    }
    ndarray_header_obj_t header;
    header.shape = shape;
    header.strides = strides;
    header.axis = axis;
    return header;
}

mp_obj_t numerical_linspace(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    static const mp_arg_t allowed_args[] = {
        { MP_QSTR_, MP_ARG_REQUIRED | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_none_obj) } },
        { MP_QSTR_, MP_ARG_REQUIRED | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_none_obj) } },
        { MP_QSTR_num, MP_ARG_INT, {.u_int = 50} },
        { MP_QSTR_endpoint, MP_ARG_KW_ONLY | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_true_obj)} },
        { MP_QSTR_retstep, MP_ARG_KW_ONLY | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_false_obj)} },
        { MP_QSTR_dtype, MP_ARG_KW_ONLY | MP_ARG_INT, {.u_int = NDARRAY_FLOAT} },
    };

    mp_arg_val_t args[MP_ARRAY_SIZE(allowed_args)];
    mp_arg_parse_all(2, pos_args, kw_args, MP_ARRAY_SIZE(allowed_args), allowed_args, args);

    uint16_t len = args[2].u_int;
    if(len < 2) {
        mp_raise_ValueError("number of points must be at least 2");
    }
    mp_float_t value, step;
    value = mp_obj_get_float(args[0].u_obj);
    uint8_t typecode = args[5].u_int;
    if(args[3].u_obj == mp_const_true) step = (mp_obj_get_float(args[1].u_obj)-value)/(len-1);
    else step = (mp_obj_get_float(args[1].u_obj)-value)/len;
    ndarray_obj_t *ndarray = ndarray_new_linear_array(len, typecode);
    if(typecode == NDARRAY_UINT8) {
        uint8_t *array = (uint8_t *)ndarray->array->items;
        for(size_t i=0; i < len; i++, value += step) array[i] = (uint8_t)value;
    } else if(typecode == NDARRAY_INT8) {
        int8_t *array = (int8_t *)ndarray->array->items;
        for(size_t i=0; i < len; i++, value += step) array[i] = (int8_t)value;
    } else if(typecode == NDARRAY_UINT16) {
        uint16_t *array = (uint16_t *)ndarray->array->items;
        for(size_t i=0; i < len; i++, value += step) array[i] = (uint16_t)value;
    } else if(typecode == NDARRAY_INT16) {
        int16_t *array = (int16_t *)ndarray->array->items;
        for(size_t i=0; i < len; i++, value += step) array[i] = (int16_t)value;
    } else {
        mp_float_t *array = (mp_float_t *)ndarray->array->items;
        for(size_t i=0; i < len; i++, value += step) array[i] = value;
    }
    if(args[4].u_obj == mp_const_false) {
        return MP_OBJ_FROM_PTR(ndarray);
    } else {
        mp_obj_t tuple[2];
        tuple[0] = ndarray;
        tuple[1] = mp_obj_new_float(step);
        return mp_obj_new_tuple(2, tuple);
    }
}

mp_obj_t numerical_flat_sum_mean_std(ndarray_obj_t *ndarray, uint8_t optype, size_t ddof) {
    mp_float_t value;
    int32_t *shape_strides = m_new(int32_t, ndarray->ndim);
    shape_strides[ndarray->ndim-1] = ndarray->strides[ndarray->ndim-1];
    for(uint8_t i=ndarray->ndim-1; i > 0; i--) {
        shape_strides[i-1] = shape_strides[i] * ndarray->shape[i-1];
    }
    if(ndarray->array->typecode == NDARRAY_UINT8) {
        CALCULATE_FLAT_SUM_STD(ndarray, uint8_t, value, shape_strides, optype);
    } else if(ndarray->array->typecode == NDARRAY_INT8) {
        CALCULATE_FLAT_SUM_STD(ndarray, int8_t, value, shape_strides, optype);
    } if(ndarray->array->typecode == NDARRAY_UINT16) {
        CALCULATE_FLAT_SUM_STD(ndarray, uint16_t, value, shape_strides, optype);
    } else if(ndarray->array->typecode == NDARRAY_INT16) {
        CALCULATE_FLAT_SUM_STD(ndarray, int16_t, value, shape_strides, optype);
    } else {
        CALCULATE_FLAT_SUM_STD(ndarray, mp_float_t, value, shape_strides, optype);
    }
    m_del(int32_t, shape_strides, ndarray->ndim);
    if(optype == NUMERICAL_SUM) {
        return mp_obj_new_float(value);
    } else if(optype == NUMERICAL_MEAN) {
        return mp_obj_new_float(value/ndarray->len);
    } else {
        return mp_obj_new_float(MICROPY_FLOAT_C_FUN(sqrt)(value/(ndarray->len-ddof)));
    }
}   

// numerical functions for ndarrays
mp_obj_t numerical_sum_mean_ndarray(ndarray_obj_t *ndarray, mp_obj_t axis, uint8_t optype) {
    if(axis == mp_const_none) {
        return numerical_flat_sum_mean_std(ndarray, optype, 0);
    } else {
        int8_t ax = mp_obj_get_int(axis);
        ndarray_header_obj_t header = contracted_shape_strides(ndarray, ax);
        ndarray_obj_t *result = ndarray_new_ndarray(ndarray->ndim-1, header.shape, header.strides, NDARRAY_FLOAT);
        mp_float_t *farray = (mp_float_t *)result->array->items;
        size_t offset;
        // iterate along the length of the output array, so as to avoid recursion
        for(size_t i=0; i < result->array->len; i++) {
            offset = ndarray_index_from_contracted(i, ndarray, result->strides, result->ndim, header.axis) + ndarray->offset;
            if(ndarray->array->typecode == NDARRAY_UINT8) {
                CALCULATE_SUM(ndarray, uint8_t, farray, ndarray->shape[header.axis], i, ndarray->strides[header.axis], offset, optype);
            } else if(ndarray->array->typecode == NDARRAY_INT8) {
                CALCULATE_SUM(ndarray, int8_t, farray, ndarray->shape[header.axis], i, ndarray->strides[header.axis], offset, optype); 
            } else if(ndarray->array->typecode == NDARRAY_UINT16) {
                CALCULATE_SUM(ndarray, uint16_t, farray, ndarray->shape[header.axis], i, ndarray->strides[header.axis], offset, optype);     
            } else if(ndarray->array->typecode == NDARRAY_INT16) {
                CALCULATE_SUM(ndarray, int16_t, farray, ndarray->shape[header.axis], i, ndarray->strides[header.axis], offset, optype);
            } else {
                CALCULATE_SUM(ndarray, mp_float_t, farray, ndarray->shape[header.axis], i, ndarray->strides[header.axis], offset, optype);       
            }
            if(optype == NUMERICAL_MEAN) farray[i] /= ndarray->shape[header.axis];
        }
        return MP_OBJ_FROM_PTR(result);
    }
    return mp_const_none;
}

mp_obj_t numerical_argmin_argmax_ndarray(ndarray_obj_t *ndarray, mp_obj_t axis, uint8_t optype) {
    return mp_const_none;
}

// numerical function for interables (single axis)
mp_obj_t numerical_argmin_argmax_iterable(mp_obj_t oin, uint8_t optype) {
    size_t idx = 0, best_idx = 0;
    mp_obj_iter_buf_t iter_buf;
    mp_obj_t iterable = mp_getiter(oin, &iter_buf);
    mp_obj_t best_obj = MP_OBJ_NULL;
    mp_obj_t item;
    mp_uint_t op = MP_BINARY_OP_LESS;
    if((optype == NUMERICAL_ARGMAX) || (optype == NUMERICAL_MAX)) op = MP_BINARY_OP_MORE;
    while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
        if ((best_obj == MP_OBJ_NULL) || (mp_binary_op(op, item, best_obj) == mp_const_true)) {
            best_obj = item;
            best_idx = idx;
        }
        idx++;
    }
    if((optype == NUMERICAL_ARGMIN) || (optype == NUMERICAL_ARGMAX)) {
        return MP_OBJ_NEW_SMALL_INT(best_idx);
    } else {
        return best_obj;
    }    
}

mp_obj_t numerical_sum_mean_std_iterable(mp_obj_t oin, uint8_t optype, size_t ddof) {
    mp_float_t value, sum = 0.0, sq_sum = 0.0;
    mp_obj_iter_buf_t iter_buf;
    mp_obj_t item, iterable = mp_getiter(oin, &iter_buf);
    mp_int_t len = mp_obj_get_int(mp_obj_len(oin));
    while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
        value = mp_obj_get_float(item);
        sum += value;
    }
    if(optype ==  NUMERICAL_SUM) {
        return mp_obj_new_float(sum);
    } else if(optype == NUMERICAL_MEAN) {
        return mp_obj_new_float(sum/len);
    } else { // this should be the case of the standard deviation
        sum /= len; // this is the mean now
        iterable = mp_getiter(oin, &iter_buf);
        while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
            value = mp_obj_get_float(item) - sum;
            sq_sum += value * value;
        }
        return mp_obj_new_float(MICROPY_FLOAT_C_FUN(sqrt)(sq_sum/(len-ddof)));
    }
}

STATIC mp_obj_t numerical_function(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args, uint8_t optype) {
    static const mp_arg_t allowed_args[] = {
        { MP_QSTR_, MP_ARG_REQUIRED | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_none_obj)} } ,
        { MP_QSTR_axis, MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_none_obj)} },
    };

    mp_arg_val_t args[MP_ARRAY_SIZE(allowed_args)];
    mp_arg_parse_all(n_args, pos_args, kw_args, MP_ARRAY_SIZE(allowed_args), allowed_args, args);
    
    mp_obj_t oin = args[0].u_obj;
    mp_obj_t axis = args[1].u_obj;
    
    if(MP_OBJ_IS_TYPE(oin, &mp_type_tuple) || MP_OBJ_IS_TYPE(oin, &mp_type_list) || 
        MP_OBJ_IS_TYPE(oin, &mp_type_range)) {
        switch(optype) {
            case NUMERICAL_MIN:
            case NUMERICAL_ARGMIN:
            case NUMERICAL_MAX:
            case NUMERICAL_ARGMAX:
                return numerical_argmin_argmax_iterable(oin, optype);
            case NUMERICAL_SUM:
            case NUMERICAL_MEAN:
                return numerical_sum_mean_std_iterable(oin, optype, 0);
            default: // we should never end up here
                return mp_const_none;
        }
    } else if(MP_OBJ_IS_TYPE(oin, &ulab_ndarray_type)) {
        ndarray_obj_t *ndarray = MP_OBJ_TO_PTR(oin);
        switch(optype) {
            case NUMERICAL_MIN:
            case NUMERICAL_ARGMIN:
            case NUMERICAL_MAX:
            case NUMERICAL_ARGMAX:
                return numerical_argmin_argmax_ndarray(ndarray, axis, optype);
            case NUMERICAL_SUM:
            case NUMERICAL_MEAN:
                return numerical_sum_mean_ndarray(ndarray, args[1].u_obj, optype);
            default:
                return mp_const_none;
        }
    } else {
        mp_raise_TypeError("input must be tuple, list, range, or ndarray");
    }
    return mp_const_none;
}

mp_obj_t numerical_sum(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    return numerical_function(n_args, pos_args, kw_args, NUMERICAL_SUM);
}

mp_obj_t numerical_mean(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    return numerical_function(n_args, pos_args, kw_args, NUMERICAL_MEAN);
}

mp_obj_t numerical_std(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    static const mp_arg_t allowed_args[] = {
        { MP_QSTR_, MP_ARG_REQUIRED | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_none_obj)} } ,
        { MP_QSTR_axis, MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_none_obj)} },
        { MP_QSTR_ddof, MP_ARG_KW_ONLY | MP_ARG_INT, {.u_int = 0} },
    };

    mp_arg_val_t args[MP_ARRAY_SIZE(allowed_args)];
    mp_arg_parse_all(n_args, pos_args, kw_args, MP_ARRAY_SIZE(allowed_args), allowed_args, args);
    
    mp_obj_t oin = args[0].u_obj;
    mp_obj_t axis = args[1].u_obj;
    size_t ddof = args[2].u_int;
    if(MP_OBJ_IS_TYPE(oin, &mp_type_tuple) || MP_OBJ_IS_TYPE(oin, &mp_type_list) || MP_OBJ_IS_TYPE(oin, &mp_type_range)) {
        return numerical_sum_mean_std_iterable(oin, NUMERICAL_STD, ddof);
    } else if(MP_OBJ_IS_TYPE(oin, &ulab_ndarray_type)) {
        ndarray_obj_t *ndarray = MP_OBJ_TO_PTR(oin);
        if(axis == mp_const_none) { // calculate for the flat array
            return numerical_flat_sum_mean_std(ndarray, NUMERICAL_STD, ddof);
        } else {
            int8_t ax = mp_obj_get_int(axis);
            ndarray_header_obj_t header = contracted_shape_strides(ndarray, ax);
            ndarray_obj_t *result = ndarray_new_ndarray(ndarray->ndim-1, header.shape, header.strides, NDARRAY_FLOAT);
            mp_float_t *farray = (mp_float_t *)result->array->items, sum_sq;
            size_t offset;
            // iterate along the length of the output array, so as to avoid recursion
            for(size_t i=0; i < result->array->len; i++) {
                offset = ndarray_index_from_contracted(i, ndarray, result->strides, result->ndim, header.axis) + ndarray->offset;
                if(ndarray->array->typecode == NDARRAY_UINT8) {
                    CALCULATE_STD(ndarray, uint8_t, sum_sq, ndarray->shape[header.axis], ndarray->strides[header.axis], offset);
                } else if(ndarray->array->typecode == NDARRAY_INT8) {
                    CALCULATE_STD(ndarray, int8_t, sum_sq, ndarray->shape[header.axis], ndarray->strides[header.axis], offset); 
                } else if(ndarray->array->typecode == NDARRAY_UINT16) {
                    CALCULATE_STD(ndarray, uint16_t, sum_sq, ndarray->shape[header.axis], ndarray->strides[header.axis], offset);     
                } else if(ndarray->array->typecode == NDARRAY_INT16) {
                    CALCULATE_STD(ndarray, int16_t, sum_sq, ndarray->shape[header.axis], ndarray->strides[header.axis], offset);
                } else {
                    CALCULATE_STD(ndarray, mp_float_t, sum_sq, ndarray->shape[header.axis], ndarray->strides[header.axis], offset);
                }
                farray[i] = MICROPY_FLOAT_C_FUN(sqrt)(sum_sq / (ndarray->shape[header.axis] - ddof));
            }
            return MP_OBJ_FROM_PTR(result);
        }
        mp_raise_TypeError("input must be tuple, list, range, or ndarray");
    }
    return mp_const_none;
}

written 14314 bytes to numerical.c


# FFT

## fft.h

In [53]:
%%ccode fft.h

#ifndef _FFT_
#define _FFT_

#ifndef MP_PI
#define MP_PI MICROPY_FLOAT_CONST(3.14159265358979323846)
#endif

#define SWAP(t, a, b) { t tmp = a; a = b; b = tmp; }

mp_obj_t fft_fft(size_t , const mp_obj_t *);
mp_obj_t fft_ifft(size_t , const mp_obj_t *);
mp_obj_t fft_spectrum(size_t , const mp_obj_t *);

#endif

written 492 bytes to fft.h


## fft.c

In [60]:
%%ccode fft.c

#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "py/runtime.h"
#include "py/binary.h"
#include "py/obj.h"
#include "py/objarray.h"
#include "ndarray.h"
#include "fft.h"

enum FFT_TYPE {
    FFT_FFT,
    FFT_IFFT,
    FFT_SPECTRUM,
};

void fft_kernel(mp_float_t *real, mp_float_t *imag, int n, int isign) {
    // This is basically a modification of four1 from Numerical Recipes
    // The main difference is that this function takes two arrays, one 
    // for the real, and one for the imaginary parts. 
    int j, m, mmax, istep;
    mp_float_t tempr, tempi;
    mp_float_t wtemp, wr, wpr, wpi, wi, theta;

    j = 0;
    for(int i = 0; i < n; i++) {
        if (j > i) {
            SWAP(mp_float_t, real[i], real[j]);
            SWAP(mp_float_t, imag[i], imag[j]);
        }
        m = n >> 1;
        while (j >= m && m > 0) {
            j -= m;
            m >>= 1;
        }
        j += m;
    }

    mmax = 1;
    while (n > mmax) {
        istep = mmax << 1;
        theta = -2.0*isign*MP_PI/istep;
        wtemp = MICROPY_FLOAT_C_FUN(sin)(0.5 * theta);
        wpr = -2.0 * wtemp * wtemp;
        wpi = MICROPY_FLOAT_C_FUN(sin)(theta);
        wr = 1.0;
        wi = 0.0;
        for(m = 0; m < mmax; m++) {
            for(int i = m; i < n; i += istep) {
                j = i + mmax;
                tempr = wr * real[j] - wi * imag[j];
                tempi = wr * imag[j] + wi * real[j];
                real[j] = real[i] - tempr;
                imag[j] = imag[i] - tempi;
                real[i] += tempr;
                imag[i] += tempi;
            }
            wtemp = wr;
            wr = wr*wpr - wi*wpi + wr;
            wi = wi*wpr + wtemp*wpi + wi;
        }
        mmax = istep;
    }
}

mp_obj_t fft_fft_ifft_spectrum(size_t n_args, mp_obj_t arg_re, mp_obj_t arg_im, uint8_t type) {
    if(!MP_OBJ_IS_TYPE(arg_re, &ulab_ndarray_type)) {
        mp_raise_NotImplementedError("FFT is defined for ndarrays only");
    } 
    if(n_args == 2) {
        if(!MP_OBJ_IS_TYPE(arg_im, &ulab_ndarray_type)) {
            mp_raise_NotImplementedError("FFT is defined for ndarrays only");
        }
    }
    // Check if input is of length of power of 2
    ndarray_obj_t *re = MP_OBJ_TO_PTR(arg_re);
    uint16_t len = re->array->len;
    if((len & (len-1)) != 0) {
        // TODO: pad the input vector, if the length is not a power of 2
        mp_raise_ValueError("input array length must be power of 2");
    }
    
    ndarray_obj_t *ndarray_re = ndarray_new_linear_array(len, NDARRAY_FLOAT);
    mp_float_t *data_re = (mp_float_t *)ndarray_re->array->items;
    
    for(size_t i=0; i < len; i++) {
        data_re[i] = ndarray_get_float_value(re->array->items, re->array->typecode, re->offset+i*re->strides[0]);
    }
    ndarray_obj_t *ndarray_im = ndarray_new_linear_array(len, NDARRAY_FLOAT);
    mp_float_t *data_im = (mp_float_t *)ndarray_im->array->items;

    if(n_args == 2) {
        ndarray_obj_t *im = MP_OBJ_TO_PTR(arg_im);
        if (re->len != im->len) {
            mp_raise_ValueError("real and imaginary parts must be of equal length");
        }
        for(size_t i=0; i < len; i++) {
            data_im[i] = ndarray_get_float_value(im->array->items, im->array->typecode, im->offset+i*im->strides[0]);
        }
    }
    if((type == FFT_FFT) || (type == FFT_SPECTRUM)) {
        fft_kernel(data_re, data_im, len, 1);
        if(type == FFT_SPECTRUM) {
            for(size_t i=0; i < len; i++) {
                data_re[i] = MICROPY_FLOAT_C_FUN(sqrt)(data_re[i]*data_re[i] + data_im[i]*data_im[i]);
            }
        }
    } else { // inverse transform
        fft_kernel(data_re, data_im, len, -1);
        // TODO: numpy accepts the norm keyword argument
        for(size_t i=0; i < len; i++) {
            data_re[i] /= len;
            data_im[i] /= len;
        }
    }
    if(type == FFT_SPECTRUM) {
        return MP_OBJ_TO_PTR(ndarray_re);
    } else {
        mp_obj_t tuple[2];
        tuple[0] = ndarray_re;
        tuple[1] = ndarray_im;
        return mp_obj_new_tuple(2, tuple);
    }
}

mp_obj_t fft_fft(size_t n_args, const mp_obj_t *args) {
    if(n_args == 2) {
        return fft_fft_ifft_spectrum(n_args, args[0], args[1], FFT_FFT);
    } else {
        return fft_fft_ifft_spectrum(n_args, args[0], mp_const_none, FFT_FFT);        
    }
}

mp_obj_t fft_ifft(size_t n_args, const mp_obj_t *args) {
    if(n_args == 2) {
        return fft_fft_ifft_spectrum(n_args, args[0], args[1], FFT_IFFT);
    } else {
        return fft_fft_ifft_spectrum(n_args, args[0], mp_const_none, FFT_IFFT);
    }
}

mp_obj_t fft_spectrum(size_t n_args, const mp_obj_t *args) {
    if(n_args == 2) {
        return fft_fft_ifft_spectrum(n_args, args[0], args[1], FFT_SPECTRUM);
    } else {
        return fft_fft_ifft_spectrum(n_args, args[0], mp_const_none, FFT_SPECTRUM);
    }
}

written 5056 bytes to fft.c


# ulab module

This module simply brings all components together, and does not contain new function definitions.

## ulab.c

In [68]:
%%ccode ulab.c

#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "py/runtime.h"
#include "py/binary.h"
#include "py/obj.h"
#include "py/objarray.h"

#include "ndarray.h"
#include "vectorise.h"
#include "linalg.h"
#include "poly.h"
#include "numerical.h"
#include "fft.h"

#define ULAB_VERSION 1.0

typedef struct _mp_obj_float_t {
    mp_obj_base_t base;
    mp_float_t value;
} mp_obj_float_t;

mp_obj_float_t ulab_version = {{&mp_type_float}, ULAB_VERSION};

MP_DEFINE_CONST_FUN_OBJ_1(ndarray_shape_obj, ndarray_shape);
MP_DEFINE_CONST_FUN_OBJ_1(ndarray_strides_obj, ndarray_strides);
MP_DEFINE_CONST_FUN_OBJ_2(ndarray_reshape_obj, ndarray_reshape);
MP_DEFINE_CONST_FUN_OBJ_1(ndarray_transpose_obj, ndarray_transpose);
MP_DEFINE_CONST_FUN_OBJ_KW(ndarray_flatten_obj, 1, ndarray_flatten);
MP_DEFINE_CONST_FUN_OBJ_1(ndarray_itemsize_obj, ndarray_itemsize);
MP_DEFINE_CONST_FUN_OBJ_1(ndarray_info_obj, ndarray_info);

MP_DEFINE_CONST_FUN_OBJ_1(vectorise_acos_obj, vectorise_acos);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_acosh_obj, vectorise_acosh);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_asin_obj, vectorise_asin);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_asinh_obj, vectorise_asinh);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_atan_obj, vectorise_atan);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_atanh_obj, vectorise_atanh);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_ceil_obj, vectorise_ceil);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_cos_obj, vectorise_cos);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_erf_obj, vectorise_erf);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_erfc_obj, vectorise_erfc);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_exp_obj, vectorise_exp);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_expm1_obj, vectorise_expm1);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_floor_obj, vectorise_floor);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_gamma_obj, vectorise_gamma);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_lgamma_obj, vectorise_lgamma);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_log_obj, vectorise_log);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_log10_obj, vectorise_log10);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_log2_obj, vectorise_log2);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_sin_obj, vectorise_sin);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_sinh_obj, vectorise_sinh);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_sqrt_obj, vectorise_sqrt);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_tan_obj, vectorise_tan);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_tanh_obj, vectorise_tanh);

MP_DEFINE_CONST_FUN_OBJ_1(linalg_inv_obj, linalg_inv);
MP_DEFINE_CONST_FUN_OBJ_2(linalg_dot_obj, linalg_dot);
MP_DEFINE_CONST_FUN_OBJ_KW(linalg_zeros_obj, 0, linalg_zeros);
MP_DEFINE_CONST_FUN_OBJ_KW(linalg_ones_obj, 0, linalg_ones);
MP_DEFINE_CONST_FUN_OBJ_KW(linalg_eye_obj, 0, linalg_eye);
MP_DEFINE_CONST_FUN_OBJ_1(linalg_det_obj, linalg_det);
MP_DEFINE_CONST_FUN_OBJ_1(linalg_eig_obj, linalg_eig);

MP_DEFINE_CONST_FUN_OBJ_2(poly_polyval_obj, poly_polyval);
MP_DEFINE_CONST_FUN_OBJ_VAR_BETWEEN(poly_polyfit_obj, 2, 3, poly_polyfit);

MP_DEFINE_CONST_FUN_OBJ_KW(numerical_linspace_obj, 2, numerical_linspace);
MP_DEFINE_CONST_FUN_OBJ_KW(numerical_sum_obj, 1, numerical_sum);
MP_DEFINE_CONST_FUN_OBJ_KW(numerical_mean_obj, 1, numerical_mean);
MP_DEFINE_CONST_FUN_OBJ_KW(numerical_std_obj, 1, numerical_std);


MP_DEFINE_CONST_FUN_OBJ_VAR_BETWEEN(fft_fft_obj, 1, 2, fft_fft);
MP_DEFINE_CONST_FUN_OBJ_VAR_BETWEEN(fft_ifft_obj, 1, 2, fft_ifft);
MP_DEFINE_CONST_FUN_OBJ_VAR_BETWEEN(fft_spectrum_obj, 1, 2, fft_spectrum);

STATIC const mp_rom_map_elem_t ulab_ndarray_locals_dict_table[] = {
    { MP_ROM_QSTR(MP_QSTR_shape), MP_ROM_PTR(&ndarray_shape_obj) },
    { MP_ROM_QSTR(MP_QSTR_strides), MP_ROM_PTR(&ndarray_strides_obj) },
    { MP_ROM_QSTR(MP_QSTR_reshape), MP_ROM_PTR(&ndarray_reshape_obj) },
    { MP_ROM_QSTR(MP_QSTR_transpose), MP_ROM_PTR(&ndarray_transpose_obj) },
    { MP_ROM_QSTR(MP_QSTR_flatten), MP_ROM_PTR(&ndarray_flatten_obj) },
    { MP_ROM_QSTR(MP_QSTR_itemsize), MP_ROM_PTR(&ndarray_itemsize_obj) },
};

STATIC MP_DEFINE_CONST_DICT(ulab_ndarray_locals_dict, ulab_ndarray_locals_dict_table);

const mp_obj_type_t ulab_ndarray_type = {
    { &mp_type_type },
    .name = MP_QSTR_ndarray,
    .print = ndarray_print,
    .make_new = ndarray_make_new,
    .subscr = ndarray_subscr,
    .getiter = ndarray_getiter,
    .unary_op = ndarray_unary_op,
    .binary_op = ndarray_binary_op,
    .locals_dict = (mp_obj_dict_t*)&ulab_ndarray_locals_dict,
};

STATIC const mp_map_elem_t ulab_globals_table[] = {
    { MP_OBJ_NEW_QSTR(MP_QSTR___name__), MP_OBJ_NEW_QSTR(MP_QSTR_ulab) },
    { MP_ROM_QSTR(MP_QSTR___version__), MP_ROM_PTR(&ulab_version) },
    { MP_OBJ_NEW_QSTR(MP_QSTR_array), (mp_obj_t)&ulab_ndarray_type },
    { MP_OBJ_NEW_QSTR(MP_QSTR_ndinfo), (mp_obj_t)&ndarray_info_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_acos), (mp_obj_t)&vectorise_acos_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_acosh), (mp_obj_t)&vectorise_acosh_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_asin), (mp_obj_t)&vectorise_asin_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_asinh), (mp_obj_t)&vectorise_asinh_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_atan), (mp_obj_t)&vectorise_atan_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_atanh), (mp_obj_t)&vectorise_atanh_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_ceil), (mp_obj_t)&vectorise_ceil_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_cos), (mp_obj_t)&vectorise_cos_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_erf), (mp_obj_t)&vectorise_erf_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_erfc), (mp_obj_t)&vectorise_erfc_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_exp), (mp_obj_t)&vectorise_exp_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_expm1), (mp_obj_t)&vectorise_expm1_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_floor), (mp_obj_t)&vectorise_floor_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_gamma), (mp_obj_t)&vectorise_gamma_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_lgamma), (mp_obj_t)&vectorise_lgamma_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_log), (mp_obj_t)&vectorise_log_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_log10), (mp_obj_t)&vectorise_log10_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_log2), (mp_obj_t)&vectorise_log2_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_sin), (mp_obj_t)&vectorise_sin_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_sinh), (mp_obj_t)&vectorise_sinh_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_sqrt), (mp_obj_t)&vectorise_sqrt_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_tan), (mp_obj_t)&vectorise_tan_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_tanh), (mp_obj_t)&vectorise_tanh_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_polyval), (mp_obj_t)&poly_polyval_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_polyfit), (mp_obj_t)&poly_polyfit_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_inv), (mp_obj_t)&linalg_inv_obj },
    { MP_ROM_QSTR(MP_QSTR_dot), (mp_obj_t)&linalg_dot_obj },
    { MP_ROM_QSTR(MP_QSTR_zeros), (mp_obj_t)&linalg_zeros_obj },
    { MP_ROM_QSTR(MP_QSTR_ones), (mp_obj_t)&linalg_ones_obj },
    { MP_ROM_QSTR(MP_QSTR_eye), (mp_obj_t)&linalg_eye_obj },
    { MP_ROM_QSTR(MP_QSTR_det), (mp_obj_t)&linalg_det_obj },
    { MP_ROM_QSTR(MP_QSTR_eig), (mp_obj_t)&linalg_eig_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_linspace), (mp_obj_t)&numerical_linspace_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_sum), (mp_obj_t)&numerical_sum_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_mean), (mp_obj_t)&numerical_mean_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_std), (mp_obj_t)&numerical_std_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_fft), (mp_obj_t)&fft_fft_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_ifft), (mp_obj_t)&fft_ifft_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_spectrum), (mp_obj_t)&fft_spectrum_obj },
    // class constants
    { MP_ROM_QSTR(MP_QSTR_bool), MP_ROM_INT(NDARRAY_BOOL) },
    { MP_ROM_QSTR(MP_QSTR_uint8), MP_ROM_INT(NDARRAY_UINT8) },
    { MP_ROM_QSTR(MP_QSTR_int8), MP_ROM_INT(NDARRAY_INT8) },
    { MP_ROM_QSTR(MP_QSTR_uint16), MP_ROM_INT(NDARRAY_UINT16) },
    { MP_ROM_QSTR(MP_QSTR_int16), MP_ROM_INT(NDARRAY_INT16) },
    { MP_ROM_QSTR(MP_QSTR_float), MP_ROM_INT(NDARRAY_FLOAT) },
};

STATIC MP_DEFINE_CONST_DICT (
    mp_module_ulab_globals,
    ulab_globals_table
);

const mp_obj_module_t ulab_user_cmodule = {
    .base = { &mp_type_module },
    .globals = (mp_obj_dict_t*)&mp_module_ulab_globals,
};

MP_REGISTER_MODULE(MP_QSTR_ulab, ulab_user_cmodule, MODULE_ULAB_ENABLED);

written 8254 bytes to ulab.c


## makefile

In [49]:
%%writefile ../../../ulab/code/micropython.mk

USERMODULES_DIR := $(USERMOD_DIR)

# Add all C files to SRC_USERMOD.
SRC_USERMOD += $(USERMODULES_DIR)/ndarray.c
SRC_USERMOD += $(USERMODULES_DIR)/vectorise.c
SRC_USERMOD += $(USERMODULES_DIR)/linalg.c
SRC_USERMOD += $(USERMODULES_DIR)/poly.c
SRC_USERMOD += $(USERMODULES_DIR)/numerical.c
SRC_USERMOD += $(USERMODULES_DIR)/fft.c
SRC_USERMOD += $(USERMODULES_DIR)/ulab.c

CFLAGS_USERMOD += -I$(USERMODULES_DIR)

Overwriting ../../../ulab/code/micropython.mk


## make

### unix port

In [49]:
%cd ../../../micropython/ports/unix/

/home/v923z/sandbox/micropython/v1.11/micropython/ports/unix


In [457]:
!make clean

Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
rm -f micropython
rm -f micropython.map
rm -rf build 


In [458]:
!make USER_C_MODULES=../../../ulab all

Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
Including User C Module from ../../../ulab/code
mkdir -p build/genhdr
GEN build/genhdr/mpversion.h
GEN build/genhdr/moduledefs.h
GEN build/genhdr/qstr.i.last
GEN build/genhdr/qstr.split
GEN build/genhdr/qstrdefs.collected.h
QSTR updated
GEN build/genhdr/qstrdefs.generated.h
mkdir -p build/build/
mkdir -p build/code/
mkdir -p build/extmod/
mkdir -p build/lib/axtls/crypto/
mkdir -p build/lib/axtls/ssl/
mkdir -p build/lib/berkeley-db-1.xx/btree/
mkdir -p build/lib/berkeley-db-1.xx/mpool/
mkdir -p build/lib/embed/
mkdir -p build/lib/mp-readline/
mkdir -p build/lib/oofatfs/
mkdir -p build/lib/timeutils/
mkdir -p build/lib/utils/
mkdir -p build/py/
CC ../../py/mpstate.c
CC ../../py/nlr.c
CC ../../py/nlrx86.c
CC ../../py/nlrx64.c
CC ../../py/nlrthumb.c
CC ../../py/nlrpowerpc.c
CC ../../py/nlrxtensa.c
CC ../../py/nlrsetjmp.c
CC ../../py/malloc.c
CC ../../py/gc.c
CC ../../py/pystack.c
CC ../../py/qstr.c
CC ../../